# 看護師離職率データセット作成（ETL完全版）

このノートブックは、手作業によるデータ作成と同等の品質（正確な数値、適切な単位）を自動化するためのものです。
特に**「沖縄県のデータ異常」「年収の単位変換」「家賃の民営借家指定」**などの重要ロジックが実装されています。

## 前提
- 元データ（CSV/Excel）が `../data/raw/` に配置されていること。
- 出力先は `../data/processed/` です。

In [2]:
import pandas as pd
import re
from pathlib import Path

RAW_DIR = Path("../data/raw")

# ==============================
# prefecture 正規化関数
# ==============================
def normalize_prefecture(x):

    if pd.isna(x):
        return x

    x = str(x)

    # 全角半角スペース削除
    x = re.sub(r"\s+", "", x)

    # 全国など除外
    if x in ["全国", "計", "未回答", "無回答・不明"]:
        return None

    # 北海道はそのまま
    if x == "北海道":
        return x

    # 接尾辞補完
    if x == "東京":
        return "東京都"

    if x == "京都":
        return "京都府"

    if x == "大阪":
        return "大阪府"

    # 都道府県が付いてない場合
    if not re.search(r"[都道府県]$", x):
        x += "県"

    return x


# ==============================
# turnover 読み込み
# ==============================
file_turnover = RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv"

df_turnover = pd.read_csv(file_turnover, encoding="utf-8-sig")

# prefecture 正規化
df_turnover["prefecture"] = df_turnover["prefecture"].apply(normalize_prefecture)

# 不要行削除
df_turnover = df_turnover.dropna(subset=["prefecture"])

# ==============================
# Data Quality Check
# ==============================
print("▼ 行数:", df_turnover.shape[0])
print("▼ 重複:", df_turnover["prefecture"].duplicated().sum())

display(df_turnover.head())


Data Loaded Successfully. Shape: (47, 4)


,prefecture,turnover_total,turnover_new_grad,turnover_experienced
0,北海道,11.5,5.9,16.6
1,青森県,8.6,10.7,16.7
2,岩手県,6.8,7.8,19.1
3,宮城県,9.1,7.1,12.4
4,秋田県,7.4,5.0,7.3


In [3]:
# ==========================
# Step 1: Master初期化 + DQチェック
# ==========================

df_master = df_turnover.copy()

print("rows:", df_master.shape[0])
print("dup prefecture:", df_master["prefecture"].duplicated().sum())

--- Data Quality Check (Step 1) ---
✅ Pass: 行数(47), ユニークキー, 欠損なし, レンジ正常


,prefecture,turnover_total,turnover_new_grad,turnover_experienced
0,北海道,11.5,5.9,16.6
1,青森県,8.6,10.7,16.7
2,岩手県,6.8,7.8,19.1
3,宮城県,9.1,7.1,12.4
4,秋田県,7.4,5.0,7.3


もし合格しなかったら（ここも手順固定）
A) rows が 47 じゃない
まず「どの行が余計/不足か」を見ます。

display(df_master["prefecture"].value_counts().head(20))

B) dup が 0 じゃない
「重複している都道府県名」を特定します。

dup_names = df_master[df_master["prefecture"].duplicated(keep=False)].sort_values("prefecture")
display(dup_names[["prefecture"]])

In [3]:
# ==========================
# Step 2: 欠損チェック（必須列が欠けていないか）
# ==========================

# 離職率ドメインの主要指標をまとめた列グループ
# ・DQチェック
# ・数値型変換
# ・レンジ検証（0〜100）
# などで共通利用する
# ※新しい離職率指標を追加した場合は必ずここに追記する
rate_cols = ["turnover_total", "turnover_new_grad", "turnover_experienced"]

print("missing counts:")
print(df_master[rate_cols].isna().sum())


## もし欠損が出た場合（その時だけ実行）
# ==========================
# Step 2b: 欠損している都道府県を特定（原因追跡の入口）
# ==========================

#for c in rate_cols:
#    miss = df_master[df_master[c].isna()][["prefecture", c]]
#   if len(miss) > 0:
#        print(f"\n--- missing in {c} ---")
#        display(miss)

missing counts:
turnover_total          0
turnover_new_grad       0
turnover_experienced    0
dtype: int64


In [4]:
# ==========================
# Step 3: レンジチェック（離職率が0〜100に収まっているか）
# ==========================

# 0未満の値を確認
print("▼ values < 0")
print((df_master[rate_cols] < 0).sum())

# 100超過の値を確認
print("\n▼ values > 100")
print((df_master[rate_cols] > 100).sum())


#もし0じゃなかった場合
#次のセルで原因を特定します。
# ==========================
# Step 3b: 範囲外の都道府県を特定
# ==========================
#
#for col in rate_cols:
#    bad = df_master[
#        (df_master[col] < 0) | (df_master[col] > 100)
#    ][[col]]
#
#    if len(bad) > 0:
#        print(f"\n--- out of range: {col} ---")
#        display(bad)

▼ values < 0
turnover_total          0
turnover_new_grad       0
turnover_experienced    0
dtype: int64

▼ values > 100
turnover_total          0
turnover_new_grad       0
turnover_experienced    0
dtype: int64


In [5]:
# ==========================
# Step 4: prefecture を主キー（index）に固定（再実行しても壊れない）
# ==========================
# 目的: 都道府県をキーに統一し、以降の join を安全にする（冪等: 何回実行しても同じ状態）

# prefecture が列にある場合だけ index 化（すでに index なら何もしない）
if "prefecture" in df_master.columns:
    df_master = df_master.set_index("prefecture")

# index状態の確認（主キーが正しく設定されているか）
print("index name:", df_master.index.name)
print("index duplicated:", df_master.index.duplicated().sum())
display(df_master.head())


index name: prefecture
index duplicated: 0


,turnover_total,turnover_new_grad,turnover_experienced
prefecture,,,
北海道,11.5,5.9,16.6
青森県,8.6,10.7,16.7
岩手県,6.8,7.8,19.1
宮城県,9.1,7.1,12.4
秋田県,7.4,5.0,7.3


In [4]:
# ==========================
# Step 2-1: night_shift を読み込んでスキーマ確認（まずは中身を見るだけ）
# ==========================
# 目的: join前に「キー列名」「値列名」「不要行の有無」を確定して事故を防ぐ

# Data Sourceのルートパスを定義
RAW_DIR = Path("../data/raw")

# 物理ファイルパスの解決（File Path Resolution）
file_night = RAW_DIR / "日本看護協会_夜勤72h超過率_都道府県別_2024.csv"
# ソースデータのインジェスト（BOM対応・DataFrameへのロード）
df_night = pd.read_csv(file_night, encoding="utf-8-sig")

# Raw Dataのプレビューによる構造（Schema）の目視確認
display(df_night.head())

# フィールド名（Column Metadata）の一覧取得
print("columns:", df_night.columns.tolist())

# データセットのボリューム（Record Count）の把握
print("rows:", df_night.shape[0])

# 各カラムのデータ型（Storage Type）の整合性確認
print("\n▼ dtypes:")
print(df_night.dtypes)

,prefecture,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift
0,北海道,36.7,7.8,4.6
1,青森県,36.5,7.7,4.8
2,岩手県,11.8,7.5,4.1
3,宮城県,30.2,8.0,4.7
4,秋田県,25.1,7.7,4.3


columns: ['prefecture', 'night_shift_72h_plus', 'night_shifts_per_month_three_shift', 'night_shifts_per_month_two_shift']
rows: 47

▼ dtypes:
prefecture                             object
night_shift_72h_plus                  float64
night_shifts_per_month_three_shift    float64
night_shifts_per_month_two_shift      float64
dtype: object


In [7]:
# prefectureキーの表記ズレがないか確認（join事故防止）
print(df_night["prefecture"].unique())

['北海道' '青森県' '岩手県' '宮城県' '秋田県' '山形県' '福島県' '茨城県' '栃木県' '群馬県' '埼玉県' '千葉県'
 '東京都' '神奈川県' '新潟県' '富山県' '石川県' '福井県' '山梨県' '長野県' '岐阜県' '静岡県' '愛知県' '三重県'
 '滋賀県' '京都府' '大阪府' '兵庫県' '奈良県' '和歌山県' '鳥取県' '島根県' '岡山県' '広島県' '山口県' '徳島県'
 '香川県' '愛媛県' '高知県' '福岡県' '佐賀県' '長崎県' '熊本県' '大分県' '宮崎県' '鹿児島県' '沖縄県']


In [8]:
# ==========================
# Step 6-1: join前チェック（列名衝突の確認）
# ==========================
# 目的: 既に同名列があると上書きや混乱が起きるため、事前に確認する

# マージ対象とする特徴量（Feature Columns）のホワイトリスト定義
night_cols = [
    "night_shift_72h_plus",
    "night_shifts_per_month_three_shift",
    "night_shifts_per_month_two_shift",
]

# ターゲット（df_master）とのカラム・コンフリクト（列名衝突）の有無を検証
print("overlap columns:", set(night_cols) & set(df_master.columns))

overlap columns: set()


In [9]:
# ==========================
# Step 6-2: night_shift を join用に整形
# ==========================
# 目的: masterと同じキー構造に揃える

# 結合キー（Join Key）をインデックスへ昇格させ、ルックアップ性能を最適化
df_night_idx = df_night.set_index("prefecture")

# インデックス・メタデータが期待通り「prefecture」であることを確認
print("index name:", df_night_idx.index.name)

# 結合時の「意図しない行の増殖（Cartesian Product）」を防ぐための重複チェック
print("index duplicated:", df_night_idx.index.duplicated().sum())

# 整形後のデータセットのカーディナリティ（行数）を最終確認
print("rows:", df_night_idx.shape[0])


index name: prefecture
index duplicated: 0
rows: 47


In [10]:
# ==========================
# Step 6-3: master に night_shift を left join（推奨テンプレ：冪等・安全）
# ==========================
# 目的: master(47都道府県)を維持したまま、夜勤系の列を最新データで入れ直す

# 統合対象とする特徴量（Feature Columns）の単一定義（SSOT: Single Source of Truth）
night_cols = [
    "night_shift_72h_plus",
    "night_shifts_per_month_three_shift",
    "night_shifts_per_month_two_shift",
]

# 結合キー（Join Key）およびユニーク性制約（Uniqueness Constraint）の事前検証
assert df_master.index.name == "prefecture", "df_master の index が prefecture ではありません"
assert df_night_idx.index.name == "prefecture", "df_night_idx の index が prefecture ではありません"
assert df_night_idx.index.duplicated().sum() == 0, "df_night_idx の prefecture が重複しています"

# 冪等性（Idempotency）確保のため、既存のターゲット列を事前にドロップ
df_master = df_master.drop(columns=night_cols, errors="ignore")
#意味: 「もし既に同じ名前の列があったら、一旦消してまっさらにする」という処理です。
#なぜやる？: これがないと、コードを2回実行したときに「同じ名前の列がもうあるよ！」とエラーで止まってしまいます。
#「何度やり直しても、常に同じ結果になる（冪等性）」ようにしておく。


# 処理前レコード件数（Baseline Record Count）のスナップショット取得
before_rows = df_master.shape[0]

# 左外部結合（Left Outer Join）によるエンティティ保持と属性拡張
df_master = df_master.join(df_night_idx[night_cols], how="left")

# 処理後レコード件数（Post-process Record Count）の取得
after_rows = df_master.shape[0]

# 整合性チェック：結合に伴う予期せぬ行増殖（Cartesian Product）の検知
if before_rows != after_rows:
    raise ValueError(f"CRITICAL: Record count changed from {before_rows} to {after_rows}")

# 実行結果のサマリー出力（Execution Logging）
print("rows before:", before_rows)
print("rows after :", after_rows)

# 結合後のデータ充填率（Fill Rate）の確認と欠損値（Missing Value）の集計
missing_counts = df_master[night_cols].isna().sum()
print("\nmissing counts (night):")
print(missing_counts)

# キー不一致（Key Mismatch）による結合失敗エンティティの特定
missing_pref = df_master[df_master[night_cols].isna().any(axis=1)].index.tolist()
print("\nmissing prefectures:", missing_pref)

# 最終的なデータフレーム・スキーマと値のプロファイリング
display(df_master.head())

rows before: 47
rows after : 47

missing counts (night):
night_shift_72h_plus                  0
night_shifts_per_month_three_shift    0
night_shifts_per_month_two_shift      0
dtype: int64

missing prefectures: []


,turnover_total,turnover_new_grad,turnover_experienced,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift
prefecture,,,,,,
北海道,11.5,5.9,16.6,36.7,7.8,4.6
青森県,8.6,10.7,16.7,36.5,7.7,4.8
岩手県,6.8,7.8,19.1,11.8,7.5,4.1
宮城県,9.1,7.1,12.4,30.2,8.0,4.7
秋田県,7.4,5.0,7.3,25.1,7.7,4.3


In [11]:
# ==========================
# Step 6-4: join後DQ（夜勤列の欠損チェック）
# ==========================
# 目的: joinに失敗した都道府県がないかを確認する

print("missing counts (night):")
print(df_master[night_cols].isna().sum())

missing_pref = df_master[df_master[night_cols].isna().any(axis=1)].index.tolist()
print("\nmissing prefectures:", missing_pref)


missing counts (night):
night_shift_72h_plus                  0
night_shifts_per_month_three_shift    0
night_shifts_per_month_two_shift      0
dtype: int64

missing prefectures: []


In [12]:
# ==========================
# Step 7: 中間成果を書き出し（Checkpoint）
# ==========================
# 目的: 統合済みmasterを保存して、次の統合作業で壊れても戻れるようにする

from pathlib import Path

# アプリケーションの出力先ディレクトリ（Target Layer）のパス定義
OUT_DIR = Path("../data/out")
# ディレクトリの冪等な作成（Recursive Directory Creation）
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 物理ファイル名およびストレージパスの解決
out_file = OUT_DIR / "master_step1_turnover_nightshift.csv"
# シリアライズ処理（BOM付きUTF-8での永続化）
df_master.to_csv(out_file, encoding="utf-8-sig")

# 処理完了のロギング（Completion Logs）
print("saved:", out_file)
# 保存済みデータセットのシェイプ（Cardinality & Schema Width）の最終記録
print("rows:", df_master.shape[0], "cols:", df_master.shape[1])

saved: ../data/out/master_step1_turnover_nightshift.csv
rows: 47 cols: 6


In [13]:
# ==========================
# Step 8: density を読み込んで構造確認
# ==========================
# 目的: join前にキー列名・粒度・型を確認する

# 物理ストレージ上のソースファイル・パスの解決（Path Resolution）
file_density = RAW_DIR / "総務省_社会生活統計指標_人口密度_2023.csv"

# 外部ソースデータからのデータ・インジェスト（Data Ingestion）
df_density = pd.read_csv(file_density, encoding="utf-8-sig")

# サンプルレコードの抽出による物理構造（Raw Structure）の目視確認
display(df_density.head())

# フィールド名（Column Metadata）の定義状況を確認
print("columns:", df_density.columns.tolist())

# データセットのカーディナリティ（データ規模とレコード件数）を把握
print("rows:", df_density.shape[0])

# 型推論（Type Inference）の結果が期待するスキーマと一致するか検証
print("\n▼ dtypes")
print(df_density.dtypes)

,統計名：,都道府県データ 社会生活統計指標,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,表番号：,10201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,表題：,Ａ 人口・世帯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,実施年月：,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,市区町村時点（年月日）：,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


columns: ['統計名：', '都道府県データ 社会生活統計指標', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32']
rows: 60

▼ dtypes
統計名：                object
都道府県データ 社会生活統計指標    object
Unnamed: 2          object
Unnamed: 3          object
Unnamed: 4          object
Unnamed: 5          object
Unnamed: 6          object
Unnamed: 7          object
Unnamed: 8          object
Unnamed: 9          object
Unnamed: 10         object
Unnamed: 11         object
Unnamed: 12         object
Unnamed: 13         object
Unnamed: 14         object
Unnamed: 15         object
Unnamed: 16         object
Unnamed: 17         object


In [14]:
# ==========================
# Step 8-1: 正しいヘッダ行を探索（先頭20行を“生”で見る）
# ==========================
# 目的: メタ情報の下にある「本当の列名行」を見つける

# データソースの物理ファイルパスを定義
file_density = RAW_DIR / "総務省_社会生活統計指標_人口密度_2023.csv"

# 自動パースを無効化し、Raw形式でインジェスト（Header Suppression）
df_density_raw = pd.read_csv(file_density, encoding="utf-8-sig", header=None)

# スキップすべき不要なメタデータ行を特定するための、データ・プロファイリング（目視確認）
display(df_density_raw.head(20))

# インジェストされたデータセットの初期ボリューム（Row/Col count）の把握
print("raw shape:", df_density_raw.shape)

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,統計名：,都道府県データ 社会生活統計指標,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,表番号：,10201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,表題：,Ａ 人口・世帯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,実施年月：,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,市区町村時点（年月日）：,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,***,調査又は集計していないもの,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-,データが得られないもの,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,X,数値が秘匿されているもの,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


raw shape: (61, 33)


In [15]:
# ==========================
# Step 8-2: density を正しいヘッダ行で読み直す
# ==========================

# 物理ストレージ上のソースファイル・パスを再定義
file_density = RAW_DIR / "総務省_社会生活統計指標_人口密度_2023.csv"

# 指定した行数（Offset）をスキップし、特定の行を論理スキーマ（Header）として採用
df_density = pd.read_csv(file_density, encoding="utf-8-sig", header=12)

# ロード済みデータセットの先頭レコードによる整合性確認（Sanity Check）
display(df_density.head())

# パースされたフィールド名（Schema Metadata）の一覧を取得
print("columns:", df_density.columns.tolist())

# インジェストされた有効なレコード件数（Active Record Count）の把握
print("rows:", df_density.shape[0])

,調査年 コード,調査年 補助コード,調査年,地域 コード,地域 補助コード,地域,/Ａ 人口・世帯,#A011000_総人口【万人】,#A0110001_総人口（男）【万人】,#A0110002_総人口（女）【万人】,...,#A03501_15歳未満人口割合【％】,#A03502_15～64歳人口割合【％】,#A03503_65歳以上人口割合【％】,#A05101_人口増減率（（A1101/A1101（-1））-1）【％】,#A05301_転入超過率（日本人移動者）【％】,#A05302_転入率（日本人移動者）【％】,#A05303_転出率（日本人移動者）【％】,#A05307_転入超過率【％】,#A05308_転入率【％】,#A05309_転出率【％】
0,2023100000,NaN,2023年度,0,NaN,全国,NaN,"12,435","6,049","6,386",...,11.4,59.5,29.1,-0.48,-,1.79,1.79,-,2.05,2.05
1,2023100000,NaN,2023年度,1000,NaN,北海道,NaN,509,241,269,...,10.1,56.9,33.0,-0.93,-0.11,0.93,1.04,-0.1,1.05,1.15
2,2023100000,NaN,2023年度,2000,NaN,青森県,NaN,118,56,63,...,10.0,54.8,35.2,-1.66,-0.47,1.29,1.76,-0.48,1.37,1.85
3,2023100000,NaN,2023年度,3000,NaN,岩手県,NaN,116,56,60,...,10.3,54.7,35.0,-1.52,-0.41,1.28,1.69,-0.4,1.39,1.79
4,2023100000,NaN,2023年度,4000,NaN,宮城県,NaN,226,111,116,...,11.1,59.7,29.2,-0.70,-0.04,1.90,1.94,-0.06,2.03,2.09


columns: ['調査年 コード', '調査年 補助コード', '調査年', '地域 コード', '地域 補助コード', '地域', '/Ａ\u3000人口・世帯', '#A011000_総人口【万人】', '#A0110001_総人口（男）【万人】', '#A0110002_総人口（女）【万人】', '#A01101_全国総人口に占める人口割合（A1101/A1101(全国)）【％】', '#A01201_総面積１km2当たり人口密度【人】', '#A01202_可住地面積１km2当たり人口密度【人】', '#A0191002_将来推計人口（2025年）【人】', '#A0191003_将来推計人口（2030年）【人】', '#A0191004_将来推計人口（2035年）【人】', '#A0191005_将来推計人口（2040年）【人】', '#A0191006_将来推計人口（2045年）【人】', '#A0191007_将来推計人口（2050年）【人】', '#A02101_人口性比（総数）（A110101/A110102）【‐】', '#A02102_人口性比（15歳未満人口）(A130101/A130102)【‐】', '#A02103_人口性比（15～64歳人口）(A130201/A130202)【‐】', '#A02104_人口性比（65歳以上人口) (A130301/A130302)【‐】', '#A03501_15歳未満人口割合【％】', '#A03502_15～64歳人口割合【％】', '#A03503_65歳以上人口割合【％】', '#A05101_人口増減率（（A1101/A1101（-1））-1）【％】', '#A05301_転入超過率（日本人移動者）【％】', '#A05302_転入率（日本人移動者）【％】', '#A05303_転出率（日本人移動者）【％】', '#A05307_転入超過率【％】', '#A05308_転入率【％】', '#A05309_転出率【％】']
rows: 48


In [16]:
# ==========================
# Step 8-3: 人口密度列を自動で特定（候補を表示）
# ==========================

# スキーマ内の全フィールドから、キーワードに合致するターゲット列を動的に抽出（Pattern Matching）
density_candidates = [c for c in df_density.columns if "人口密度" in str(c)]

# 自動特定された候補カラムのリストを検証用にロギング（Discovery Results）
print("density_candidates:", density_candidates)

density_candidates: ['#A01201_総面積１km2当たり人口密度【人】', '#A01202_可住地面積１km2当たり人口密度【人】']


In [17]:
# ==========================
# Step 8-4: density（総面積ベース）を整形して master に冪等 join
# ==========================
# 目的: prefectureをキーに、人口密度（総面積1km2あたり）をmasterへ追加する（再実行しても壊れない）

# 特定されたターゲット指標の論理カラム名を物理定義
density_col = "#A01201_総面積１km2当たり人口密度【人】"

# アウトオブスコープ（全国）の除外と、サブセットの抽出（Column Selection）
df_density_clean = df_density.loc[df_density["地域"] != "全国", ["地域", density_col]].copy()

# 下流工程（Downstream）のスキーマ標準に合わせたリネーム（Canonical Mapping）
df_density_clean = df_density_clean.rename(columns={"地域": "prefecture", density_col: "density"})

# データの正規化：型変換の妨げとなるノイズ（カンマ・記号）のクリーニング
df_density_clean["density"] = (
    df_density_clean["density"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .replace({"-": None, "***": None, "X": None})
)
# 強制的な型変換（Coercion）による数値データ型へのキャストと欠損値処理
df_density_clean["density"] = pd.to_numeric(df_density_clean["density"], errors="coerce")

# 結合演算の効率化に向けたインデックス化とユニーク制約の検証
df_density_idx = df_density_clean.set_index("prefecture")
print("dup density index:", df_density_idx.index.duplicated().sum())
print("rows density:", df_density_idx.shape[0])

# 冪等性（Idempotency）の確保：既存属性のクリーンアップ
df_master = df_master.drop(columns=["density"], errors="ignore")

# 結合前後のレコード件数比較（Baseline Comparison）
before_rows = df_master.shape[0]
# 左外部結合（Left Outer Join）によるマスタの属性拡張（Feature Enrichment）
df_master = df_master.join(df_density_idx[["density"]], how="left")
after_rows = df_master.shape[0]

# データ整合性バリデーション：行数変化の有無を確認
if before_rows != after_rows:
    raise ValueError(f"CRITICAL: Record count changed from {before_rows} to {after_rows}")

# パイプライン実行ログの出力
print("rows before:", before_rows)
print("rows after :", after_rows)

# 結合成功率（Coverage Check）の算出と欠損分析
print("\nmissing counts (density):")
print(df_master[["density"]].isna().sum())

# キーのミスマッチ（Unmatched Keys）が発生しているエンティティの特定
missing_pref = df_master[df_master["density"].isna()].index.tolist()
print("missing prefectures:", missing_pref)

# 最終的なデータ構造と分布のプロファイリング
display(df_master.head())

# ==========================
# Step 8-4: 人口密度データの統合（作業メモ）
# ==========================
# 人口密度データの「地域」列を、マスタ側と合わせるために「prefecture」へとリネームした。
# データに混入していた「全国」などの集計行は、分析の邪魔になるため事前に除外してある。
# 数字に含まれるカンマや「-」などの記号をクリーニングし、計算ができるよう数値型へ変換した。
# その際、変換できない不正な値はエラーにせず、安全に欠損値（NaN）として処理している。
# 結合時にデータが重複して増えないよう、都道府県をインデックスに設定して一意性を確保した。
# コードを何度再実行してもエラーにならないよう、既存の列を一度削除してから結合する設計にした。
# マスタにある47都道府県の行を削らないよう、Left Join（左外部結合）で人口密度を統合。
# 結合の前後で行数が変わっていないかを検証し、データが壊れていないことを保証した。
# 最後に、マスタ側の県名と一致せず結合に失敗した箇所がないか、欠損数を数えて確認した。

dup density index: 0
rows density: 47
rows before: 47
rows after : 47

missing counts (density):
density    0
dtype: int64
missing prefectures: []


,turnover_total,turnover_new_grad,turnover_experienced,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift,density
prefecture,,,,,,,
北海道,11.5,5.9,16.6,36.7,7.8,4.6,64.9
青森県,8.6,10.7,16.7,36.5,7.7,4.8,122.8
岩手県,6.8,7.8,19.1,11.8,7.5,4.1,76.1
宮城県,9.1,7.1,12.4,30.2,8.0,4.7,310.9
秋田県,7.4,5.0,7.3,25.1,7.7,4.3,78.5


In [18]:
# ==========================
# Step 8-5: Checkpoint保存（turnover + night + density）
# ==========================

# 出力先ディレクトリが存在しない場合に備えた、ディレクトリの動的生成
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 中間成果物（Artifact）の物理保存パスの定義
checkpoint_path = OUT_DIR / "master_step2_density.csv"

# インデックスの列復元（Flattening）を行い、ポータビリティの高い形式でCSV出力
df_master.reset_index().to_csv(
    checkpoint_path,
    index=False,
    encoding="utf-8-sig"
)

# 保存完了のロギング（Persistence Confirmation）
print("saved:", checkpoint_path)
# 出力されたデータセットの次元（Cardinality & Schema Width）の最終確認
print("rows:", df_master.shape[0], "cols:", df_master.shape[1])

saved: ../data/out/master_step2_density.csv
rows: 47 cols: 7


In [19]:
# ==========================
# Step 8-6: density → population_density に正式名称統一
# ==========================

# フィールド名の曖昧さを排除し、ドメイン知識に基づいた標準名称へリネーム（Semantic Labeling）
df_master = df_master.rename(columns={
    "density": "population_density"
})

# 修正後のスキーマ（Schema）に定義の齟齬がないか最終確認
print("columns:", df_master.columns.tolist())

# カラム名変更後のデータセットの物理状態をプレビュー（Final Inspection）
display(df_master.head())

columns: ['turnover_total', 'turnover_new_grad', 'turnover_experienced', 'night_shift_72h_plus', 'night_shifts_per_month_three_shift', 'night_shifts_per_month_two_shift', 'population_density']


,turnover_total,turnover_new_grad,turnover_experienced,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift,population_density
prefecture,,,,,,,
北海道,11.5,5.9,16.6,36.7,7.8,4.6,64.9
青森県,8.6,10.7,16.7,36.5,7.7,4.8,122.8
岩手県,6.8,7.8,19.1,11.8,7.5,4.1,76.1
宮城県,9.1,7.1,12.4,30.2,8.0,4.7,310.9
秋田県,7.4,5.0,7.3,25.1,7.7,4.3,78.5


In [20]:
# ==========================
# Step 9-1: job_openings_ratio（有効求人倍率）読み込み（唯一の入口セル）
# ==========================
file_job = RAW_DIR / "厚労省_一般職業紹介状況_有効求人倍率_2025.xlsx"
sheet_job = "第１８表ー４　有効求人倍率（実数）"

df_job = pd.read_excel(file_job, sheet_name=sheet_job, header=1)

# ここで最低限のスキーマ確認（壊れてたら先に止まる）
print("loaded sheet:", sheet_job)
print("rows:", df_job.shape[0], "cols:", df_job.shape[1])
print("西暦 in columns?", "西暦" in df_job.columns)
print("first 5 columns:", df_job.columns.tolist()[:5])

# 目視：月次ブロックがある付近（環境により多少ズレてもOK）
display(df_job.iloc[255:265, :5])

loaded sheet: 第１８表ー４　有効求人倍率（実数）
rows: 290 cols: 50
西暦 in columns? True
first 5 columns: ['西暦', '和暦', 'Unnamed: 2', '北海道', '青森県']


,西暦,和暦,Unnamed: 2,北海道,青森県
255,2023年,令和５年,１月,1.17,1.26
256,2023年,令和５年,２月,1.12,1.24
257,2023年,令和５年,３月,1.14,1.34
258,2023年,令和５年,４月,1.11,1.28
259,2023年,令和５年,５月,1.05,1.27
260,2023年,令和５年,６月,1.12,1.26
261,2023年,令和５年,７月,1.18,1.28
262,2023年,令和５年,８月,1.19,1.32
263,2023年,令和５年,９月,1.18,1.38
264,2023年,令和５年,10月,1.19,1.45


In [21]:
# ==========================
# Step 9-2: 年(西暦) + 月(Unnamed: 2) から _ym を生成
# ==========================
import re

month_col = "Unnamed: 2"  # このシートは3列目が月（例: "６月"）

# 前提チェック（ここで落ちるのが正しい）
if "西暦" not in df_job.columns:
    raise KeyError("df_job に '西暦' 列がありません。Step 9-1 の読み込みを確認してください。")
if month_col not in df_job.columns:
    raise KeyError(f"df_job に '{month_col}' 列がありません。列名: {df_job.columns.tolist()[:10]}")

def parse_month_num(x):
    """'６月' / '10月' / '１月' などから月(1-12)を取り出す"""
    if pd.isna(x):
        return None
    s = str(x).strip()
    s = s.translate(str.maketrans("０１２３４５６７８９", "0123456789"))  # 全角→半角
    m = re.search(r"(\d{1,2})", s)
    return int(m.group(1)) if m else None

# 年（例: 2023年）→ 数値
df_job["_year"] = (
    df_job["西暦"].astype(str)
    .str.replace("年", "", regex=False)
    .str.strip()
)
df_job["_year"] = pd.to_numeric(df_job["_year"], errors="coerce")

# 月（例: ６月）→ 数値
df_job["_month"] = df_job[month_col].apply(parse_month_num)

# 年月合成（年 or 月が欠ける行はNaT）
df_job["_ym"] = pd.to_datetime(
    dict(year=df_job["_year"], month=df_job["_month"], day=1),
    errors="coerce"
)

print("month rows:", df_job["_ym"].notna().sum())
print("min:", df_job["_ym"].min(), "max:", df_job["_ym"].max())
display(df_job.loc[df_job["_ym"].notna(), ["西暦", month_col, "_ym"]].tail(15))


month rows: 250
min: 2005-02-01 00:00:00 max: 2025-11-01 00:00:00


,西暦,Unnamed: 2,_ym
275,2024年,９月,2024-09-01
276,2024年,10月,2024-10-01
277,2024年,11月,2024-11-01
278,2024年,12月,2024-12-01
279,2025年,１月,2025-01-01
280,2025年,２月,2025-02-01
281,2025年,３月,2025-03-01
282,2025年,４月,2025-04-01
283,2025年,５月,2025-05-01
284,2025年,６月,2025-06-01


In [22]:
# ==========================
# Step 9-3: 2023年度（2023/04〜2024/03）抽出 + 12ヶ月チェック
# ==========================
start = pd.Timestamp(2023, 4, 1)
end   = pd.Timestamp(2024, 3, 1)

df_fy = df_job[df_job["_ym"].between(start, end)].copy()

months = sorted(df_fy["_ym"].dropna().unique())
print("months found:", len(months))
print("months list:", [d.strftime("%Y-%m") for d in months])

if len(months) != 12:
    raise ValueError("2023年度の月が12ヶ月揃っていません。シート構造/欠損を確認してください。")


months found: 12
months list: ['2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12', '2024-01', '2024-02', '2024-03']


In [23]:
# ==========================
# Step 9-4: 生データ（Raw Data）のインジェストと物理構造のプロファイリング
# ==========================

# Rawデータ・インジェスト：スキーマ推論を介在させず、ソースの全情報を生の状態でロード開始
df_job_raw = pd.read_excel(
    file_job,  # ソースパス定義：不変的なファイルパスを参照し、データソースへのコネクタとして機能
    sheet_name="第１８表ー４　有効求人倍率（実数）",  # サブセット指定：マルチシートから特定のドメインデータを抽出対象に特定
    header=None  # スキーマ保護：Excel特有の多段ヘッダーによる誤判定を防ぎ、物理的な全レコードをRawとしてキャプチャ
)

# 初期プロファイリング：ヘッダーのオフセット（実データの開始位置）や、データの汚れを特定するための目視検品
display(df_job_raw.head(15))

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,都道府県（就業地）別有効求人倍率（実数）（パートタイムを含む一般）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,西暦,和暦,NaN,北海道,青森県,岩手県,宮城県,秋田県,山形県,福島県,...,愛媛県,高知県,福岡県,佐賀県,長崎県,熊本県,大分県,宮崎県,鹿児島県,沖縄県
2,NaN,NaN,NaN,倍,倍,倍,倍,倍,倍,倍,...,倍,倍,倍,倍,倍,倍,倍,倍,倍,倍
3,2006年,平成18年,NaN,0.59,0.45,0.79,0.86,0.65,1.13,0.94,...,0.93,0.51,0.78,0.78,0.61,0.84,1.08,0.73,0.62,0.41
4,2007年,平成19年,NaN,0.57,0.48,0.77,0.89,0.65,1.02,0.93,...,0.92,0.52,0.79,0.84,0.64,0.85,1.12,0.71,0.63,0.43
5,2008年,平成20年,NaN,0.45,0.44,0.6,0.67,0.53,0.82,0.71,...,0.88,0.49,0.62,0.67,0.59,0.64,0.91,0.61,0.54,0.4
6,2009年,平成21年,NaN,0.37,0.29,0.36,0.39,0.32,0.39,0.37,...,0.56,0.4,0.41,0.46,0.44,0.4,0.5,0.43,0.38,0.3
7,2010年,平成22年,NaN,0.42,0.37,0.44,0.43,0.44,0.53,0.43,...,0.62,0.48,0.45,0.52,0.49,0.49,0.56,0.48,0.45,0.32
8,2011年,平成23年,NaN,0.47,0.46,0.57,0.65,0.55,0.67,0.63,...,0.77,0.58,0.56,0.65,0.6,0.64,0.7,0.61,0.57,0.31
9,2012年,平成24年,NaN,0.59,0.62,0.94,1.11,0.71,0.93,1.09,...,0.84,0.61,0.68,0.76,0.69,0.72,0.78,0.74,0.68,0.42


In [24]:
# ==========================
# Step 9-4: 都道府県別 2023年度平均（job_openings_ratio）を作成
# ==========================

# 都道府県列を抽出（列名が「○○都/道/府/県」で終わる列）
pref_cols = [c for c in df_job.columns if isinstance(c, str) and c.endswith(("都","道","府","県"))]
print("pref cols:", len(pref_cols))

if len(pref_cols) != 47:
    raise ValueError(f"都道府県列が47個ではありません: {len(pref_cols)}")

tmp = df_fy[pref_cols].apply(pd.to_numeric, errors="coerce")
job_ratio_mean = tmp.mean(axis=0)  # 列ごと平均（都道府県ごと）

df_job_fy = job_ratio_mean.reset_index()
df_job_fy.columns = ["prefecture", "job_openings_ratio"]

print("rows job:", df_job_fy.shape[0])
print("missing job_openings_ratio:", df_job_fy["job_openings_ratio"].isna().sum())
display(df_job_fy.head())


pref cols: 47
rows job: 47
missing job_openings_ratio: 0


,prefecture,job_openings_ratio
0,北海道,1.120833
1,青森県,1.305833
2,岩手県,1.328333
3,宮城県,1.339167
4,秋田県,1.476667


In [25]:
# ==========================
# Step 9-5: master に冪等 join（job_openings_ratio）
# ==========================

# join用index
df_job_idx = df_job_fy.set_index("prefecture")
print("dup job index:", df_job_idx.index.duplicated().sum())

# 冪等（再実行耐性）：既存列を削除して入れ直し
df_master = df_master.drop(columns=["job_openings_ratio"], errors="ignore")

before_rows = df_master.shape[0]
df_master = df_master.join(df_job_idx[["job_openings_ratio"]], how="left")
after_rows = df_master.shape[0]

# 行増殖チェック
if before_rows != after_rows:
    raise ValueError(f"CRITICAL: Record count changed from {before_rows} to {after_rows}")

print("rows before:", before_rows)
print("rows after :", after_rows)

# 欠損チェック
print("\nmissing counts (job_openings_ratio):")
print(df_master[["job_openings_ratio"]].isna().sum())

missing_pref = df_master[df_master["job_openings_ratio"].isna()].index.tolist()
print("missing prefectures:", missing_pref)

display(df_master.head())


dup job index: 0
rows before: 47
rows after : 47

missing counts (job_openings_ratio):
job_openings_ratio    0
dtype: int64
missing prefectures: []


,turnover_total,turnover_new_grad,turnover_experienced,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift,population_density,job_openings_ratio
prefecture,,,,,,,,
北海道,11.5,5.9,16.6,36.7,7.8,4.6,64.9,1.120833
青森県,8.6,10.7,16.7,36.5,7.7,4.8,122.8,1.305833
岩手県,6.8,7.8,19.1,11.8,7.5,4.1,76.1,1.328333
宮城県,9.1,7.1,12.4,30.2,8.0,4.7,310.9,1.339167
秋田県,7.4,5.0,7.3,25.1,7.7,4.3,78.5,1.476667


In [26]:
# ==========================
# Step 10-1: rent_private（民営家賃）データのインジェストと構造定義
# ==========================

# ソースパス定義：不変的なRawデータ層の定数を参照し、対象ソースへのセキュアなアクセスパスを構築
rent_path = RAW_DIR / "総務省_住宅土地統計調査_家賃_2024.xlsx"

# 抽出（Extraction）：多段ヘッダー情報を保持したまま、バイナリを構造化データへ変換開始
df_rent_raw = pd.read_excel(
    rent_path,          # ターゲット指定：定義済みのパスオブジェクトを渡し、I/O処理の入力ソースを確定
    sheet_name="e112_2",# サブセット特定：ブック内の特定ドメイン（家賃統計シート）をインジェスト対象に限定
    header=[4,5,6]      # 階層スキーマ定義：マルチインデックスを直接キャプチャし、データのセマンティクスを保護
)

# ボリューム検証：ロード後の行列サイズを確認し、レコード欠落やスキーマ定義の誤りがないか検証
print(df_rent_raw.shape)

# 物理構造プロファイリング：階層ヘッダーの展開状態を視覚的に検品し、後続の変換ロジックを精査
df_rent_raw.head()

(5197, 19)


Unnamed: 0_level_0 Unnamed: 1_level_0                 表章項目   借家（専用住宅）数  \
  Unnamed: 0_level_1 Unnamed: 1_level_1                  事項名 住宅の１か月当たり家賃   
  Unnamed: 0_level_2 Unnamed: 1_level_2                  項目名       00_総数   
0                NaN                NaN                 表章単位           戸   
1            地域識別コード               地域区分                住宅の種類               
2                  a           00000_全国                 0_総数    19399600   
3                  a           00000_全国             11_公営の借家     1759700   
4                  a           00000_全国  12_都市再生機構(UR)・公社の借家      715800   

                                                                  \
                                                                   
    01_0円 02_1～10,000円未満 03_10,000～20,000円未満 04_20,000～40,000円未満   
0       戸              戸                   戸                   戸   
1     NaN            NaN                 NaN                 NaN   
2  308500         421300              973100             3212000   
3   19000         197800              562500              751700   
4     100            200                1100               88500   

                                                                \
                                                                 
  05_40,000～60,000円未満 06_60,000～80,000円未満 07_80,000～100,000円未満   
0                   戸                   戸                    戸   
1                 NaN                 NaN                  NaN   
2             5852000             4689500              1832700   
3              140300               55300                17800   
4              246100              153200                95200   

                                                                     \
                                                                      
  08_100,000～150,000円未満 09_150,000～200,000円未満 10_200,000円以上   99_不詳   
0                     戸                     戸             戸       戸   
1                   NaN                   NaN           NaN     NaN   
2               1444600                310300        151800  204000   
3                 11300                  1300           400    2400   
4                102600                 23500          5500       0   

  住宅の１か月当たり家賃             １か月当たり共益費・管理費            
     住宅の家賃の平均             住宅の共益費・管理費の平均            
    1_家賃０円を含む 2_家賃０円を含まない       1_０円を含む 2_０円を含まない  
0           円           円             円         円  
1         NaN         NaN           NaN       NaN  
2       59656       60630          3037      4806  
3       24961       25234          1706      2197  
4       71831       71842          3353      3787

In [27]:
# ==========================
# Step 10-2: マルチインデックスのフラットニングとカラム名の正規化
# ==========================

# スキーマ・リマッピング：ダウンストリームでの参照互換性を高めるため、階層型ヘッダーを単一レイヤーへ集約
df_rent_raw.columns = [
    # セマンティック・コンカリネーション：各階層のラベルを連結し、特徴量の意味的な文脈を文字列として保存
    "_".join([str(c) for c in col if str(c) != "nan"])
    # 物理名サニタイズ：Excel特有の空階層（nan）を除去し、ノイズのないクリーンな物理名を構築
    for col in df_rent_raw.columns
]

# スキーマ・カタログの最終確認：変換後の物理名をリスト化し、後続のクレンジング工程の定義体として利用
df_rent_raw.columns.tolist()

['Unnamed: 0_level_0_Unnamed: 0_level_1_Unnamed: 0_level_2',
 'Unnamed: 1_level_0_Unnamed: 1_level_1_Unnamed: 1_level_2',
 '表章項目_事項名_項目名',
 '借家（専用住宅）数_住宅の１か月当たり家賃_00_総数',
 '借家（専用住宅）数_住宅の１か月当たり家賃_01_0円',
 '借家（専用住宅）数_住宅の１か月当たり家賃_02_1～10,000円未満',
 '借家（専用住宅）数_住宅の１か月当たり家賃_03_10,000～20,000円未満',
 '借家（専用住宅）数_住宅の１か月当たり家賃_04_20,000～40,000円未満',
 '借家（専用住宅）数_住宅の１か月当たり家賃_05_40,000～60,000円未満',
 '借家（専用住宅）数_住宅の１か月当たり家賃_06_60,000～80,000円未満',
 '借家（専用住宅）数_住宅の１か月当たり家賃_07_80,000～100,000円未満',
 '借家（専用住宅）数_住宅の１か月当たり家賃_08_100,000～150,000円未満',
 '借家（専用住宅）数_住宅の１か月当たり家賃_09_150,000～200,000円未満',
 '借家（専用住宅）数_住宅の１か月当たり家賃_10_200,000円以上',
 '借家（専用住宅）数_住宅の１か月当たり家賃_99_不詳',
 '住宅の１か月当たり家賃_住宅の家賃の平均_1_家賃０円を含む',
 '住宅の１か月当たり家賃_住宅の家賃の平均_2_家賃０円を含まない',
 '１か月当たり共益費・管理費_住宅の共益費・管理費の平均_1_０円を含む',
 '１か月当たり共益費・管理費_住宅の共益費・管理費の平均_2_０円を含まない']

In [28]:
# ==========================
# Step 10-3: 特徴量探索（Feature Discovery）とメタデータ・スキャン
# ==========================

# スキーマ・走査（Scanning）：正規化された全カラム名をイテレーションし、特徴量の候補を網羅的に探索
for c in df_rent_raw.columns:
    # 属性フィルタリング：特定のビジネスロジック（平均指標）を内包するカラムを、部分一致によりヒューリスティックに特定
    if "平均" in c:
        # メタデータ・インスペクション：特定された物理名をプロファイリングし、サブセット定義用（カタログ作成）のリストを確定
        print(c)

住宅の１か月当たり家賃_住宅の家賃の平均_1_家賃０円を含む
住宅の１か月当たり家賃_住宅の家賃の平均_2_家賃０円を含まない
１か月当たり共益費・管理費_住宅の共益費・管理費の平均_1_０円を含む
１か月当たり共益費・管理費_住宅の共益費・管理費の平均_2_０円を含まない


In [29]:
# ===== 固定列名（ここはハードコードでOK）=====
COL_AREA = "Unnamed: 1_level_0_Unnamed: 1_level_1_Unnamed: 1_level_2"   # 00000_全国 / 01000_北海道 / 01100_札幌市...
COL_HOUSE_TYPE = "表章項目_事項名_項目名"                                # 13_民営借家 など

COL_RENT_EXCL0 = "住宅の１か月当たり家賃_住宅の家賃の平均_2_家賃０円を含まない"
COL_FEE_EXCL0  = "１か月当たり共益費・管理費_住宅の共益費・管理費の平均_2_０円を含まない"

df_rent = df_rent_raw[[COL_AREA, COL_HOUSE_TYPE, COL_RENT_EXCL0, COL_FEE_EXCL0]].copy()

# --- 都道府県(01000〜47000)だけ残す（00000全国や01100札幌市などを落とす）---
df_rent = df_rent[df_rent[COL_AREA].astype(str).str.match(r"^\d{5}_.+")].copy()
df_rent["code"] = df_rent[COL_AREA].astype(str).str.slice(0, 5).astype(int)

df_rent = df_rent[(df_rent["code"] % 1000 == 0) & (df_rent["code"] >= 1000) & (df_rent["code"] <= 47000)].copy()

# --- 民営借家だけ ---
df_rent = df_rent[df_rent[COL_HOUSE_TYPE].astype(str).str.startswith("13_民営借家")].copy()

print("rows after filter:", len(df_rent))
df_rent.head()

rows after filter: 47


,Unnamed: 1_level_0_Unnamed: 1_level_1_Unnamed: 1_level_2,表章項目_事項名_項目名,住宅の１か月当たり家賃_住宅の家賃の平均_2_家賃０円を含まない,１か月当たり共益費・管理費_住宅の共益費・管理費の平均_2_０円を含まない,code
10,01000_北海道,13_民営借家,51199,4700,1000
240,02000_青森県,13_民営借家,46340,3996,2000
295,03000_岩手県,13_民営借家,49350,3776,3000
370,04000_宮城県,13_民営借家,56735,4425,4000
470,05000_秋田県,13_民営借家,47534,3422,5000


In [30]:
def normalize_prefecture_name_from_area(area: str) -> str:
    if pd.isna(area):
        return None
    name = str(area).split("_", 1)[1]
    name = re.sub(r"\s+", "", name)
    if name == "東京": name = "東京都"
    if name == "大阪": name = "大阪府"
    if name == "京都": name = "京都府"
    return name

def to_num(x):
    return pd.to_numeric(x, errors="coerce")

df_rent["prefecture"] = df_rent[COL_AREA].apply(normalize_prefecture_name_from_area)
df_rent["avg_rent_excl0"] = to_num(df_rent[COL_RENT_EXCL0])
df_rent["avg_fee_excl0"]  = to_num(df_rent[COL_FEE_EXCL0])

print("rows:", len(df_rent))
print("dup prefecture:", df_rent["prefecture"].duplicated().sum())
print("null prefecture:", df_rent["prefecture"].isna().sum())
print("null avg_rent_excl0:", df_rent["avg_rent_excl0"].isna().sum())
print("null avg_fee_excl0:", df_rent["avg_fee_excl0"].isna().sum())

df_rent[["prefecture","avg_rent_excl0","avg_fee_excl0"]].head()


rows: 47
dup prefecture: 0
null prefecture: 0
null avg_rent_excl0: 0
null avg_fee_excl0: 0


,prefecture,avg_rent_excl0,avg_fee_excl0
10,北海道,51199,4700
240,青森県,46340,3996
295,岩手県,49350,3776
370,宮城県,56735,4425
470,秋田県,47534,3422


In [31]:
df_rent["rent_private"] = df_rent["avg_rent_excl0"] + df_rent["avg_fee_excl0"]

df_rent_out = df_rent[["prefecture","rent_private","avg_rent_excl0","avg_fee_excl0"]].copy()
df_rent_out.head()


,prefecture,rent_private,avg_rent_excl0,avg_fee_excl0
10,北海道,55899,51199,4700
240,青森県,50336,46340,3996
295,岩手県,53126,49350,3776
370,宮城県,61160,56735,4425
470,秋田県,50956,47534,3422


In [32]:
print("rows:", len(df_rent_out))
print("dup prefecture:", df_rent_out["prefecture"].duplicated().sum())
print("null rent_private:", df_rent_out["rent_private"].isna().sum())

print("\nsummary rent_private:")
print(df_rent_out["rent_private"].describe())

print("\nTop 5 highest:")
display(df_rent_out.sort_values("rent_private", ascending=False).head(5))

print("\nTop 5 lowest:")
display(df_rent_out.sort_values("rent_private", ascending=True).head(5))


rows: 47
dup prefecture: 0
null rent_private: 0

summary rent_private:
count        47.000000
mean      58644.531915
std        9188.922165
min       50034.000000
25%       53789.500000
50%       55199.000000
75%       60631.000000
max      101117.000000
Name: rent_private, dtype: float64

Top 5 highest:


,prefecture,rent_private,avg_rent_excl0,avg_fee_excl0
1460,東京都,101117,94802,6315
1715,神奈川県,81603,76356,5247
3595,兵庫県,71143,65097,6046
1240,千葉県,70876,65813,5063
3270,大阪府,70487,64263,6224



Top 5 lowest:


,prefecture,rent_private,avg_rent_excl0,avg_fee_excl0
4990,宮崎県,50034,46290,3744
240,青森県,50336,46340,3996
5040,鹿児島県,50791,46916,3875
470,秋田県,50956,47534,3422
4915,大分県,51207,47163,4044


In [33]:
from pathlib import Path

# 冪等：既に列があれば落としてからmerge
if "rent_private" in df_master.columns:
    df_master = df_master.drop(columns=["rent_private"])

before = len(df_master)

df_master = df_master.merge(
    df_rent_out[["prefecture","rent_private"]],
    on="prefecture",
    how="left",
    validate="one_to_one"
)

after = len(df_master)

print("before rows:", before, "after rows:", after)
print("null rent_private:", df_master["rent_private"].isna().sum())

before rows: 47 after rows: 47
null rent_private: 0


In [34]:
# checkpoint保存
OUT_DIR = Path("../data/out")
out_path = OUT_DIR / "master_step4_housing_rent.csv"
df_master.to_csv(out_path, index=False, encoding="utf-8-sig")
print("saved:", out_path)

df_master.head()

saved: ../data/out/master_step4_housing_rent.csv


,prefecture,turnover_total,turnover_new_grad,turnover_experienced,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift,population_density,job_openings_ratio,rent_private
0,北海道,11.5,5.9,16.6,36.7,7.8,4.6,64.9,1.120833,55899
1,青森県,8.6,10.7,16.7,36.5,7.7,4.8,122.8,1.305833,50336
2,岩手県,6.8,7.8,19.1,11.8,7.5,4.1,76.1,1.328333,53126
3,宮城県,9.1,7.1,12.4,30.2,8.0,4.7,310.9,1.339167,61160
4,秋田県,7.4,5.0,7.3,25.1,7.7,4.3,78.5,1.476667,50956


In [35]:
# ==========================
# Step 11-1: ソース読み込み（持ち家率Excel）と表構造の確定
#   - sheet名
#   - ヘッダ位置（多段か）
#   - 都道府県キー列（01000形式か）
# ==========================

RAW_DIR = Path("../data/raw")  # データレイク（Raw層）の格納ルートディレクトリを定義
owner_path = RAW_DIR / "総務省_住宅土地統計調査_持ち家率_2024.xlsx"  # インジェスト対象ファイルの絶対パス解決

xls = pd.ExcelFile(owner_path)  # メタデータ参照用にExcelFileオブジェクトをインスタンス化（全量ロード回避によるメモリ最適化）
print("sheets:", xls.sheet_names)  # シート構成のバリデーション（期待するシートが存在するかログ出力）

# まず1枚目を覗く
sheet = xls.sheet_names[0]  # 処理対象となるプライマリシートの抽出

df_peek_owner = pd.read_excel(owner_path, sheet_name=sheet, header=None)  # ヘッダ位置判定のため、スキーマレス（header=None）でRawデータをロード
display(df_peek_owner.head(25))  # データ構造およびダーティデータ特定のためのプロファイリング（サンプリング確認）

sheets: ['令和５年住宅・土地統計調査\u3000住宅及び世帯に関する基本集 (2)']


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,令和５年住宅・土地統計調査 住宅及び世帯に関する基本集計,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,第３－２表 住宅の所有の関係(2区分)別住宅数並びに世帯の種類(3区分)別世帯数及び世帯人員...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,表章項目,住宅数,世帯数,世帯数,世帯数,世帯数,世帯数,世帯人員,世帯人員,世帯人員,世帯人員,世帯人員
5,NaN,NaN,事項名,NaN,世帯の種類,世帯の種類,世帯の種類,世帯の種類,世帯の種類,世帯の種類,世帯の種類,世帯の種類,世帯の種類,世帯の種類
6,NaN,NaN,項目名,NaN,0_総数,1_主世帯,11_１人世帯,12_２人以上の世帯,2_同居世帯,0_総数,1_主世帯,11_１人世帯,12_２人以上の世帯,2_同居世帯
7,NaN,NaN,表章単位,戸,世帯,世帯,世帯,世帯,世帯,人,人,人,人,人
8,地域識別コード,地域区分,住宅の所有の関係,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,a,00000_全国,0_総数,55665000,56071400,55665000,21449000,34216100,406400,121773900,121017200,21449000,99568300,756700


In [36]:
# ==========================
# Step 11-2：読み込み（MultiIndex）→ フラット化
# ==========================

from pathlib import Path
import pandas as pd

RAW_DIR = Path("../data/raw")
owner_path = RAW_DIR / "総務省_住宅土地統計調査_持ち家率_2024.xlsx"
SHEET_OWNER = "令和５年住宅・土地統計調査　住宅及び世帯に関する基本集 (2)"

# rentと同じ：ヘッダは 4,5,6 行
df_owner_raw = pd.read_excel(owner_path, sheet_name=SHEET_OWNER, header=[4,5,6])

# フラット化
df_owner_raw.columns = [
    "_".join([str(c) for c in col if str(c) != "nan"])
    for col in df_owner_raw.columns
]

print(df_owner_raw.shape)
df_owner_raw.head()



(3851, 14)


,Unnamed: 0_level_0_Unnamed: 0_level_1_Unnamed: 0_level_2,Unnamed: 1_level_0_Unnamed: 1_level_1_Unnamed: 1_level_2,表章項目_事項名_項目名,住宅数_Unnamed: 3_level_1_Unnamed: 3_level_2,世帯数_世帯の種類_0_総数,世帯数_世帯の種類_1_主世帯,世帯数_世帯の種類_11_１人世帯,世帯数_世帯の種類_12_２人以上の世帯,世帯数_世帯の種類_2_同居世帯,世帯人員_世帯の種類_0_総数,世帯人員_世帯の種類_1_主世帯,世帯人員_世帯の種類_11_１人世帯,世帯人員_世帯の種類_12_２人以上の世帯,世帯人員_世帯の種類_2_同居世帯
0,NaN,NaN,表章単位,戸,世帯,世帯,世帯,世帯,世帯,人,人,人,人,人
1,地域識別コード,地域区分,住宅の所有の関係,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a,00000_全国,0_総数,55665000,56071400,55665000,21449000,34216100,406400,121773900,121017200,21449000,99568300,756700
3,a,00000_全国,1_持ち家,33875500,34169800,33875500,7715700,26159900,294300,86285800,85679200,7715700,77963500,606600
4,a,00000_全国,2_借家,19461700,19573800,19461700,11849000,7612700,112100,32249700,32099600,11849000,20250600,150000


In [37]:
# ==========================
# Step 11-3：必要列の特定（主世帯列を固定）
# ==========================

# 「世帯数 × 主世帯」を含む列を探す（確認用）
for c in df_owner_raw.columns:
    if "世帯数" in c and "主世帯" in c:
        print(c)

# ↑ 出力された列名をそのまま貼る（ここが仕様固定ポイント）
COL_HOUSEHOLD_MAIN = "世帯数_世帯の種類_1_主世帯"

# 安全チェック（存在しなければここで停止）
assert COL_HOUSEHOLD_MAIN in df_owner_raw.columns, (
    f"主世帯列が見つかりません: {COL_HOUSEHOLD_MAIN}"
)

print("OK：主世帯列を固定しました")


世帯数_世帯の種類_1_主世帯
OK：主世帯列を固定しました


In [38]:
# ==========================
# Step 11-4：都道府県 × 所有区分（0_総数 / 1_持ち家）抽出
# ==========================

# 必須カラム（rentと同じ構造）
COL_AREA = "Unnamed: 1_level_0_Unnamed: 1_level_1_Unnamed: 1_level_2"  # 01000_北海道 など
COL_OWN  = "表章項目_事項名_項目名"                                      # 0_総数 / 1_持ち家 / 2_借家

# 安全チェック
assert COL_AREA in df_owner_raw.columns, f"missing {COL_AREA}"
assert COL_OWN  in df_owner_raw.columns, f"missing {COL_OWN}"
assert COL_HOUSEHOLD_MAIN in df_owner_raw.columns, f"missing {COL_HOUSEHOLD_MAIN}"

# 必要列だけ抽出
df_owner = df_owner_raw[[COL_AREA, COL_OWN, COL_HOUSEHOLD_MAIN]].copy()

# 都道府県のみ（01000〜47000）を残す
df_owner = df_owner[df_owner[COL_AREA].astype(str).str.match(r"^\d{5}_.+")].copy()
df_owner["code"] = df_owner[COL_AREA].astype(str).str.slice(0, 5).astype(int)

df_owner = df_owner[
    (df_owner["code"] % 1000 == 0) &
    (df_owner["code"] >= 1000) &
    (df_owner["code"] <= 47000)
].copy()

# 所有区分は「0_総数」「1_持ち家」だけ
df_owner = df_owner[df_owner[COL_OWN].isin(["0_総数", "1_持ち家"])].copy()

print("rows:", len(df_owner))   # 期待：94（47×2）
df_owner.head()


rows: 94


,Unnamed: 1_level_0_Unnamed: 1_level_1_Unnamed: 1_level_2,表章項目_事項名_項目名,世帯数_世帯の種類_1_主世帯,code
5,01000_北海道,0_総数,2423200,1000
6,01000_北海道,1_持ち家,1381200,1000
185,02000_青森県,0_総数,488700,2000
186,02000_青森県,1_持ち家,349100,2000
230,03000_岩手県,0_総数,476700,3000


In [39]:
# ==========================
# Step 11-5：主世帯ベースで持ち家率を算出
# ==========================

# 都道府県名を正規化
def normalize_prefecture_name_from_area(area: str) -> str:
    if pd.isna(area):
        return None
    name = str(area).split("_", 1)[1]
    name = re.sub(r"\s+", "", name)
    if name == "東京": name = "東京都"
    if name == "大阪": name = "大阪府"
    if name == "京都": name = "京都府"
    return name

# prefecture 列を作成
df_owner["prefecture"] = df_owner[COL_AREA].apply(normalize_prefecture_name_from_area)

# 数値化（安全のため）
df_owner["households_main"] = pd.to_numeric(
    df_owner[COL_HOUSEHOLD_MAIN],
    errors="coerce"
)

# wide化：0_総数 と 1_持ち家 を横に並べる
df_owner_wide = (
    df_owner
    .pivot_table(
        index="prefecture",
        columns=COL_OWN,
        values="households_main",
        aggfunc="sum"
    )
    .reset_index()
)

# 持ち家率（%）を計算
df_owner_wide["home_ownership_rate"] = (
    df_owner_wide["1_持ち家"] / df_owner_wide["0_総数"]
) * 100

# 出力用
df_owner_out = df_owner_wide[["prefecture", "home_ownership_rate"]].copy()
df_owner_out.head()

# ==========================
# pivot結果が47になっているか確認
# ==========================

print("df_owner rows:", len(df_owner))  # 94のはず
print("unique prefectures:", df_owner["prefecture"].nunique())
print("unique COL_OWN:", df_owner[COL_OWN].unique())

# 欠損チェック（ここが0でないと pivot で落ちる）
print("null prefecture:", df_owner["prefecture"].isna().sum())
print("null households_main:", df_owner["households_main"].isna().sum())

# pivot後の形
print("df_owner_wide shape:", df_owner_wide.shape)
print("df_owner_wide columns:", df_owner_wide.columns.tolist())
df_owner_wide.head()



df_owner rows: 94
unique prefectures: 47
unique COL_OWN: ['0_総数' '1_持ち家']
null prefecture: 0
null households_main: 0
df_owner_wide shape: (47, 4)
df_owner_wide columns: ['prefecture', '0_総数', '1_持ち家', 'home_ownership_rate']


表章項目_事項名_項目名,prefecture,0_総数,1_持ち家,home_ownership_rate
0,三重県,727300,526100,72.336037
1,京都府,1182900,717700,60.672922
2,佐賀県,311900,211800,67.906380
3,兵庫県,2397400,1545000,64.444815
4,北海道,2423200,1381200,56.999010


In [40]:
# ==========================
# Step 11-6：df_masterへ持ち家率を統合（冪等join）＋ checkpoint保存
# ==========================

from pathlib import Path

# ---- 冪等処理：既に列があれば削除 ----
if "home_ownership_rate" in df_master.columns:
    df_master = df_master.drop(columns=["home_ownership_rate"])

before_rows = len(df_master)

# ---- join ----
df_master = df_master.merge(
    df_owner_out,
    on="prefecture",
    how="left",
    validate="one_to_one"
)

after_rows = len(df_master)

# ---- DQチェック ----
print("before rows:", before_rows)
print("after rows:", after_rows)
print("null home_ownership_rate:", df_master["home_ownership_rate"].isna().sum())

# ---- checkpoint保存 ----
OUT_DIR = Path("../data/out")
out_path = OUT_DIR / "master_step5_housing_owner.csv"

df_master.to_csv(out_path, index=False, encoding="utf-8-sig")

print("saved:", out_path)
df_master.head()


before rows: 47
after rows: 47
null home_ownership_rate: 0
saved: ../data/out/master_step5_housing_owner.csv


,prefecture,turnover_total,turnover_new_grad,turnover_experienced,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift,population_density,job_openings_ratio,rent_private,home_ownership_rate
0,北海道,11.5,5.9,16.6,36.7,7.8,4.6,64.9,1.120833,55899,56.999010
1,青森県,8.6,10.7,16.7,36.5,7.7,4.8,122.8,1.305833,50336,71.434418
2,岩手県,6.8,7.8,19.1,11.8,7.5,4.1,76.1,1.328333,53126,70.253828
3,宮城県,9.1,7.1,12.4,30.2,8.0,4.7,310.9,1.339167,61160,59.975607
4,秋田県,7.4,5.0,7.3,25.1,7.7,4.3,78.5,1.476667,50956,77.097997


In [41]:
# ==========================
# Step 12-1：データ構造の把握
# ==========================
RAW_DIR = Path("../data/raw")
commute_path = RAW_DIR / "総務省_社会生活基本調査_通勤時間_2022.xlsx"

xls = pd.ExcelFile(commute_path)
print("sheets:", xls.sheet_names)

df_peek_commute = pd.read_excel(
    commute_path,
    sheet_name=xls.sheet_names[0],
    header=None
)

df_peek_commute.head(25)


sheets: ['1日の生活時間の使い方から', '1年間の活動から']


,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,社会生活基本調査から分かる47都道府県ランキング （令和３年社会生活基本調査結果より）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,【1日の生活時間の使い方から 】,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,睡眠時間たっぷり！？ランキング,NaN,NaN,NaN,NaN,早起き！？ランキング,NaN,NaN,NaN,...,NaN,NaN,スローライフ！？たっぷり食事時間\nランキング,NaN,NaN,NaN,NaN,イクメン！？ランキング,NaN,NaN
3,NaN,順位,都道府県名,時間.分,NaN,NaN,順位,都道府県名,平均時刻,NaN,...,NaN,NaN,順位,都道府県名,時間.分,NaN,NaN,順位,都道府県名,時間.分
4,NaN,NaN,全国平均,7.54,NaN,NaN,NaN,全国平均,6:38,NaN,...,NaN,NaN,NaN,全国平均,1.39,NaN,NaN,NaN,全国平均,1.54
5,NaN,1,青森県,8.08,NaN,NaN,1,青森県,06:17:00,NaN,...,NaN,NaN,1,山梨県,1.45,NaN,NaN,1,奈良県,2.35
6,NaN,2,秋田県,8.06,NaN,NaN,2,岩手県,06:21:00,NaN,...,NaN,NaN,1,長野県,1.45,NaN,NaN,2,新潟県,2.33
7,NaN,3,鹿児島県,8.05,NaN,NaN,2,秋田県,06:21:00,NaN,...,NaN,NaN,3,秋田県,1.44,NaN,NaN,3,高知県,2.27
8,NaN,4,宮城県,8.04,NaN,NaN,4,長野県,06:22:00,NaN,...,NaN,NaN,4,奈良県,1.43,NaN,NaN,4,和歌山県,2.21
9,NaN,4,高知県,8.04,NaN,NaN,5,富山県,06:24:00,NaN,...,NaN,NaN,5,茨城県,1.42,NaN,NaN,5,千葉県,2.2


In [42]:
# ==========================
# Step 12-2：通勤時間の原表を探す
# ==========================

import pandas as pd
from pathlib import Path

RAW_DIR = Path("../data/raw")
commute_path = RAW_DIR / "総務省_社会生活基本調査_通勤時間_2022.xlsx"

xls = pd.ExcelFile(commute_path)

for sheet in xls.sheet_names:
    print("\n--- sheet:", sheet, "---")
    df_peek = pd.read_excel(commute_path, sheet_name=sheet, header=None)
    display(df_peek.head(15))



--- sheet: 1日の生活時間の使い方から ---


,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,社会生活基本調査から分かる47都道府県ランキング （令和３年社会生活基本調査結果より）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,【1日の生活時間の使い方から 】,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,睡眠時間たっぷり！？ランキング,NaN,NaN,NaN,NaN,早起き！？ランキング,NaN,NaN,NaN,...,NaN,NaN,スローライフ！？たっぷり食事時間\nランキング,NaN,NaN,NaN,NaN,イクメン！？ランキング,NaN,NaN
3,NaN,順位,都道府県名,時間.分,NaN,NaN,順位,都道府県名,平均時刻,NaN,...,NaN,NaN,順位,都道府県名,時間.分,NaN,NaN,順位,都道府県名,時間.分
4,NaN,NaN,全国平均,7.54,NaN,NaN,NaN,全国平均,6:38,NaN,...,NaN,NaN,NaN,全国平均,1.39,NaN,NaN,NaN,全国平均,1.54
5,NaN,1,青森県,8.08,NaN,NaN,1,青森県,06:17:00,NaN,...,NaN,NaN,1,山梨県,1.45,NaN,NaN,1,奈良県,2.35
6,NaN,2,秋田県,8.06,NaN,NaN,2,岩手県,06:21:00,NaN,...,NaN,NaN,1,長野県,1.45,NaN,NaN,2,新潟県,2.33
7,NaN,3,鹿児島県,8.05,NaN,NaN,2,秋田県,06:21:00,NaN,...,NaN,NaN,3,秋田県,1.44,NaN,NaN,3,高知県,2.27
8,NaN,4,宮城県,8.04,NaN,NaN,4,長野県,06:22:00,NaN,...,NaN,NaN,4,奈良県,1.43,NaN,NaN,4,和歌山県,2.21
9,NaN,4,高知県,8.04,NaN,NaN,5,富山県,06:24:00,NaN,...,NaN,NaN,5,茨城県,1.42,NaN,NaN,5,千葉県,2.2



--- sheet: 1年間の活動から ---


,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,社会生活基本調査から分かる47都道府県ランキング （令和３年社会生活基本調査結果より）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,【1年間の活動から 】,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,ウォーキングが人気！？ランキング,NaN,NaN,NaN,NaN,トレーニングが大好き！？ランキング,NaN,NaN,NaN,...,NaN,NaN,バスケが人気！？ランキング,NaN,NaN,NaN,NaN,つり人が多い！？ランキング,NaN,NaN
3,NaN,順位,都道府県名,行動者率（％）,NaN,NaN,順位,都道府県名,行動者率（％）,NaN,...,NaN,NaN,順位,都道府県名,行動者率（％）,NaN,NaN,順位,都道府県名,行動者率（％）
4,NaN,NaN,全国平均,44.3,NaN,NaN,NaN,全国平均,12.9,NaN,...,NaN,NaN,NaN,全国平均,3.6,NaN,NaN,NaN,全国平均,7.8
5,NaN,1,東京都,52.3,NaN,NaN,1,東京都,15,NaN,...,NaN,NaN,1,秋田県,5.3,NaN,NaN,1,広島県,12.2
6,NaN,2,神奈川県,49.3,NaN,NaN,1,神奈川県,15,NaN,...,NaN,NaN,2,沖縄県,5.1,NaN,NaN,2,愛媛県,11.9
7,NaN,3,埼玉県,47.4,NaN,NaN,3,埼玉県,14.4,NaN,...,NaN,NaN,3,島根県,4.9,NaN,NaN,3,熊本県,11.8
8,NaN,4,千葉県,46.9,NaN,NaN,4,大阪府,14,NaN,...,NaN,NaN,4,宮城県,4.7,NaN,NaN,4,山口県,11.2
9,NaN,5,群馬県,45.8,NaN,NaN,4,福岡県,14,NaN,...,NaN,NaN,5,福岡県,4.5,NaN,NaN,5,長崎県,10.9


In [43]:
# ==========================
# Step 12-3：Q列の変数定義を確認
# ==========================

sheet = "1日の生活時間の使い方から"

df_check = pd.read_excel(
    commute_path,
    sheet_name=sheet,
    header=None
)

# Q列（index=16）周辺を確認
df_check.iloc[0:10, 15:20]


,15,16,17,18,19
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,通勤・通学時間が長い！？\nランキング,NaN,NaN,NaN
3,NaN,順位,都道府県名,時間.分,NaN
4,NaN,NaN,全国平均,1.19,NaN
5,NaN,1,神奈川県,1.4,NaN
6,NaN,2,千葉県,1.35,NaN
7,NaN,2,東京都,1.35,NaN
8,NaN,4,埼玉県,1.34,NaN
9,NaN,5,奈良県,1.28,NaN


In [44]:
# ==========================
# Step 12-4：通勤・通学（ランキング表）が47件揃っているか確認
# ==========================

import pandas as pd
import numpy as np

sheet = "1日の生活時間の使い方から"
df = pd.read_excel(commute_path, sheet_name=sheet, header=None)

COL_RANK = 16  # Q列: 見出し「通勤・通学時間が長い！？ランキング」
# このブロックのヘッダ行は 3（順位/都道府県名/時間.分）
header_row = 3

# ヘッダ位置（列）を特定：都道府県名 / 時間.分
pref_col = 17  # R列
time_col = 18  # S列

df_block = df.loc[header_row+1:, [pref_col, time_col]].copy()
df_block.columns = ["prefecture", "time_hm"]

# 全国平均行などを除外（都道府県だけ残す）
df_block = df_block[df_block["prefecture"].astype(str).str.endswith(("都","道","府","県"))].copy()

print("rows(prefectures):", len(df_block))
print("unique prefectures:", df_block["prefecture"].nunique())
df_block.head(10)


rows(prefectures): 47
unique prefectures: 47


,prefecture,time_hm
5,神奈川県,1.4
6,千葉県,1.35
7,東京都,1.35
8,埼玉県,1.34
9,奈良県,1.28
10,大阪府,1.27
11,兵庫県,1.24
12,京都府,1.21
13,茨城県,1.18
14,愛知県,1.18


In [45]:
# ==========================
# Step 12-5：通勤・通学時間を分に正規化
# ==========================

import pandas as pd
import numpy as np

# 時間.分（例: 1.35）→ 分 に変換する関数
def time_hm_to_minutes(x):
    """
    時間.分 形式（例: 1.35 = 1時間35分）を分に変換する
    """
    if pd.isna(x):
        return np.nan
    x = float(x)
    hours = int(x)
    minutes = round((x - hours) * 100)
    return hours * 60 + minutes

# 変換
df_block["commute_time_minutes"] = df_block["time_hm"].apply(time_hm_to_minutes)

# DQチェック
print("rows:", len(df_block))
print("null minutes:", df_block["commute_time_minutes"].isna().sum())
print(df_block["commute_time_minutes"].describe())

df_block.head(10)


rows: 47
null minutes: 0
count     47.000000
mean      69.893617
std       11.037946
min       56.000000
25%       62.500000
50%       66.000000
75%       74.000000
max      100.000000
Name: commute_time_minutes, dtype: float64


,prefecture,time_hm,commute_time_minutes
5,神奈川県,1.4,100
6,千葉県,1.35,95
7,東京都,1.35,95
8,埼玉県,1.34,94
9,奈良県,1.28,88
10,大阪府,1.27,87
11,兵庫県,1.24,84
12,京都府,1.21,81
13,茨城県,1.18,78
14,愛知県,1.18,78


In [46]:
# ==========================
# Step 12-6：df_masterへ通勤時間を統合（冪等join）＋ checkpoint保存
# ==========================

from pathlib import Path

# join用の最終データ（prefecture + minutes）
df_commute_out = df_block[["prefecture", "commute_time_minutes"]].copy()

# ---- 冪等処理：既に列があれば削除 ----
if "commute_time_minutes" in df_master.columns:
    df_master = df_master.drop(columns=["commute_time_minutes"])

before_rows = len(df_master)

# ---- join ----
df_master = df_master.merge(
    df_commute_out,
    on="prefecture",
    how="left",
    validate="one_to_one"
)

after_rows = len(df_master)

# ---- DQチェック ----
print("before rows:", before_rows)
print("after rows:", after_rows)
print("null commute_time_minutes:", df_master["commute_time_minutes"].isna().sum())

# ---- checkpoint保存 ----
OUT_DIR = Path("../data/out")
out_path = OUT_DIR / "master_step6_housing_commute.csv"

df_master.to_csv(out_path, index=False, encoding="utf-8-sig")

print("saved:", out_path)
df_master.head()


before rows: 47
after rows: 47
null commute_time_minutes: 0
saved: ../data/out/master_step6_housing_commute.csv


,prefecture,turnover_total,turnover_new_grad,turnover_experienced,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift,population_density,job_openings_ratio,rent_private,home_ownership_rate,commute_time_minutes
0,北海道,11.5,5.9,16.6,36.7,7.8,4.6,64.9,1.120833,55899,56.999010,64
1,青森県,8.6,10.7,16.7,36.5,7.7,4.8,122.8,1.305833,50336,71.434418,61
2,岩手県,6.8,7.8,19.1,11.8,7.5,4.1,76.1,1.328333,53126,70.253828,63
3,宮城県,9.1,7.1,12.4,30.2,8.0,4.7,310.9,1.339167,61160,59.975607,73
4,秋田県,7.4,5.0,7.3,25.1,7.7,4.3,78.5,1.476667,50956,77.097997,60


In [47]:
# ==========================
# Step 13-1：住宅3点の最終DQ（型・レンジ・欠損）
# ==========================

cols = ["rent_private", "home_ownership_rate", "commute_time_minutes"]

print("rows:", len(df_master))
print("\nnulls:")
print(df_master[cols].isna().sum())

print("\ndtypes:")
print(df_master[cols].dtypes)

print("\ndescribe:")
print(df_master[cols].describe())

# 異常値チェック（レンジ）
bad_owner = df_master[(df_master["home_ownership_rate"] < 0) | (df_master["home_ownership_rate"] > 100)]
bad_commute = df_master[(df_master["commute_time_minutes"] < 0) | (df_master["commute_time_minutes"] > 180)]
bad_rent = df_master[(df_master["rent_private"] < 10000) | (df_master["rent_private"] > 200000)]

print("\nout of range owner:", len(bad_owner))
print("out of range commute:", len(bad_commute))
print("out of range rent:", len(bad_rent))


rows: 47

nulls:
rent_private            0
home_ownership_rate     0
commute_time_minutes    0
dtype: int64

dtypes:
rent_private              int64
home_ownership_rate     float64
commute_time_minutes      int64
dtype: object

describe:
        rent_private  home_ownership_rate  commute_time_minutes
count      47.000000            47.000000             47.000000
mean    58644.531915            66.110613             69.893617
std      9188.922165             7.277841             11.037946
min     50034.000000            42.572522             56.000000
25%     53789.500000            63.104868             62.500000
50%     55199.000000            67.844066             66.000000
75%     60631.000000            70.981238             74.000000
max    101117.000000            77.097997            100.000000

out of range owner: 0
out of range commute: 0
out of range rent: 0


In [48]:
# ==========================
# Step 13-2：住宅3点の最終確定（型固定 → master保存）
# ==========================

from pathlib import Path
import numpy as np

# 型固定（再現性のために明示）
df_master["rent_private"] = df_master["rent_private"].astype("int64")
df_master["commute_time_minutes"] = df_master["commute_time_minutes"].astype("int64")

# rateは小数が意味を持つので小数2桁に丸めて固定（見た目＆再現性）
df_master["home_ownership_rate"] = df_master["home_ownership_rate"].round(2)

# 保存（住宅確定版）
OUT_DIR = Path("../data/out")
out_path = OUT_DIR / "master_step6_housing_final.csv"
df_master.to_csv(out_path, index=False, encoding="utf-8-sig")

print("saved:", out_path)
df_master[["prefecture","rent_private","home_ownership_rate","commute_time_minutes"]].head()


saved: ../data/out/master_step6_housing_final.csv


,prefecture,rent_private,home_ownership_rate,commute_time_minutes
0,北海道,55899,57.00,64
1,青森県,50336,71.43,61
2,岩手県,53126,70.25,63
3,宮城県,61160,59.98,73
4,秋田県,50956,77.10,60


In [49]:
# ==========================
# Step 14-1：住宅確定版masterの復元（起点）
# ==========================

from pathlib import Path
import pandas as pd

OUT_DIR = Path("../data/out")
master_path = OUT_DIR / "master_step6_housing_final.csv"

df_master = pd.read_csv(master_path, encoding="utf-8-sig")

print("rows:", len(df_master))
print("dup prefecture:", df_master["prefecture"].duplicated().sum())
print(df_master.head(3))


rows: 47
dup prefecture: 0
  prefecture  turnover_total  turnover_new_grad  turnover_experienced  \
0        北海道            11.5                5.9                  16.6   
1        青森県             8.6               10.7                  16.7   
2        岩手県             6.8                7.8                  19.1   

   night_shift_72h_plus  night_shifts_per_month_three_shift  \
0                  36.7                                 7.8   
1                  36.5                                 7.7   
2                  11.8                                 7.5   

   night_shifts_per_month_two_shift  population_density  job_openings_ratio  \
0                               4.6                64.9            1.120833   
1                               4.8               122.8            1.305833   
2                               4.1                76.1            1.328333   

   rent_private  home_ownership_rate  commute_time_minutes  
0         55899                57.00             

In [50]:
# ==========================
# Step 14-2：看護師数CSVの構造確認
# ==========================


RAW_DIR = Path("../data/raw")
nurse_path = RAW_DIR / "厚労省_衛生行政報告例_看護師数_2023.csv"

# 衛生行政報告例は cp932 + header=3 が多い
df_nurse_raw = pd.read_csv(nurse_path, encoding="cp932", header=3)

print("shape:", df_nurse_raw.shape)
print("\ncolumns:")
for c in df_nurse_raw.columns:
    print("-", c)

print("\nhead:")
display(df_nurse_raw.head(5))


shape: (50, 9)

columns:
- Unnamed: 0
- 実数
- 実数.1
- 実数.2
- 実数.3
- 率（人口１０万対）
- 率（人口１０万対）.1
- 率（人口１０万対）.2
- 率（人口１０万対）.3

head:


,Unnamed: 0,実数,実数.1,実数.2,実数.3,率（人口１０万対）,率（人口１０万対）.1,率（人口１０万対）.2,率（人口１０万対）.3
0,NaN,保健師,助産師,看護師,准看護師,保健師,助産師,看護師,准看護師
1,NaN,人,人,人,人,NaN,NaN,NaN,NaN
2,全国,60299,38063,1311687,254329,48.3,30.5,1049.8,203.5
3,北海道,3288,1571,67176,13065,64,30.6,1306.9,254.2
4,青森県,709,340,13463,4374,58.9,28.2,1118.2,363.3


In [51]:
# ==========================
# Step 14-2-①：看護職員データを正しいヘッダ位置で読み込み（df_staffを作る）
# ==========================

RAW_DIR = Path("../data/raw")
staff_path = RAW_DIR / "厚労省_衛生行政報告例_看護師数_2023.csv"

# まずは想定どおり header=3 で読む（あなたのheadの形とも整合）
df_staff_raw = pd.read_csv(staff_path, encoding="cp932", header=3)

print("shape:", df_staff_raw.shape)
print("columns:", df_staff_raw.columns.tolist())
display(df_staff_raw.head(6))

# 先頭2行（区分名・単位）を落として、データ行だけにする
df_staff = df_staff_raw.iloc[2:].copy()

print("\n(df_staff) shape:", df_staff.shape)
display(df_staff.head(5))


shape: (50, 9)
columns: ['Unnamed: 0', '実数', '実数.1', '実数.2', '実数.3', '率（人口１０万対）', '率（人口１０万対）.1', '率（人口１０万対）.2', '率（人口１０万対）.3']


,Unnamed: 0,実数,実数.1,実数.2,実数.3,率（人口１０万対）,率（人口１０万対）.1,率（人口１０万対）.2,率（人口１０万対）.3
0,NaN,保健師,助産師,看護師,准看護師,保健師,助産師,看護師,准看護師
1,NaN,人,人,人,人,NaN,NaN,NaN,NaN
2,全国,60299,38063,1311687,254329,48.3,30.5,1049.8,203.5
3,北海道,3288,1571,67176,13065,64,30.6,1306.9,254.2
4,青森県,709,340,13463,4374,58.9,28.2,1118.2,363.3
5,岩手県,831,394,14383,2479,70.4,33.4,1217.9,209.9



(df_staff) shape: (48, 9)


,Unnamed: 0,実数,実数.1,実数.2,実数.3,率（人口１０万対）,率（人口１０万対）.1,率（人口１０万対）.2,率（人口１０万対）.3
2,全国,60299,38063,1311687,254329,48.3,30.5,1049.8,203.5
3,北海道,3288,1571,67176,13065,64,30.6,1306.9,254.2
4,青森県,709,340,13463,4374,58.9,28.2,1118.2,363.3
5,岩手県,831,394,14383,2479,70.4,33.4,1217.9,209.9
6,宮城県,1165,771,21304,4643,51.1,33.8,934.4,203.6


In [52]:
# ==========================
# Step 14-3：看護職員（実数）整形 → 合算列作成（＋看護師のみ）
# ==========================

import pandas as pd

# 列名を固定（読みやすさ＆事故防止）
df_staff2 = df_staff.rename(columns={
    "Unnamed: 0": "prefecture",
    "実数": "public_health_nurse",
    "実数.1": "midwife",
    "実数.2": "nurse",
    "実数.3": "assistant_nurse",
}).copy()

# 全国行を除外（都道府県47にする）
df_staff2 = df_staff2[df_staff2["prefecture"] != "全国"].copy()

# 数値化（実数4職種）
for col in ["public_health_nurse", "midwife", "nurse", "assistant_nurse"]:
    df_staff2[col] = pd.to_numeric(df_staff2[col], errors="coerce")

# 合算（看護職員：保健師・助産師・看護師・准看護師）
df_staff2["nurse_total_staff"] = (
    df_staff2["public_health_nurse"]
    + df_staff2["midwife"]
    + df_staff2["nurse"]
    + df_staff2["assistant_nurse"]
)

# 好奇心枠：看護師のみ
df_staff2["nurse_only"] = df_staff2["nurse"]

# join用に必要列だけ
df_staff_out = df_staff2[["prefecture", "nurse_total_staff", "nurse_only"]].copy()

# DQチェック
print("rows:", len(df_staff_out))
print("dup prefecture:", df_staff_out["prefecture"].duplicated().sum())
print("null nurse_total_staff:", df_staff_out["nurse_total_staff"].isna().sum())
print("null nurse_only:", df_staff_out["nurse_only"].isna().sum())

df_staff_out.head()


rows: 47
dup prefecture: 0
null nurse_total_staff: 0
null nurse_only: 0


,prefecture,nurse_total_staff,nurse_only
3,北海道,85100,67176
4,青森県,18886,13463
5,岩手県,18087,14383
6,宮城県,27883,21304
7,秋田県,15267,11767


In [53]:
# ==========================
# Step 14-4：df_masterへ看護職員数を統合（冪等join）＋ checkpoint保存
# ==========================

from pathlib import Path

# 冪等処理：既に列があれば削除
for col in ["nurse_total_staff", "nurse_only"]:
    if col in df_master.columns:
        df_master = df_master.drop(columns=[col])

before_rows = len(df_master)

# join
df_master = df_master.merge(
    df_staff_out,
    on="prefecture",
    how="left",
    validate="one_to_one"
)

after_rows = len(df_master)

# DQチェック
print("before rows:", before_rows)
print("after rows:", after_rows)
print("null nurse_total_staff:", df_master["nurse_total_staff"].isna().sum())
print("null nurse_only:", df_master["nurse_only"].isna().sum())

# checkpoint保存
OUT_DIR = Path("../data/out")
out_path = OUT_DIR / "master_step7_medical_staff.csv"
df_master.to_csv(out_path, index=False, encoding="utf-8-sig")

print("saved:", out_path)
df_master[["prefecture", "nurse_total_staff", "nurse_only"]].head()


before rows: 47
after rows: 47
null nurse_total_staff: 0
null nurse_only: 0
saved: ../data/out/master_step7_medical_staff.csv


,prefecture,nurse_total_staff,nurse_only
0,北海道,85100,67176
1,青森県,18886,13463
2,岩手県,18087,14383
3,宮城県,27883,21304
4,秋田県,15267,11767


In [54]:
# ==========================
# Step 15-1：看護師年収データ（Excel）の構造把握
# ==========================

from pathlib import Path
import pandas as pd

RAW_DIR = Path("../data/raw")
income_path = RAW_DIR / "厚労省_賃金構造基本統計調査_看護師年収_2024.xlsx"

xls = pd.ExcelFile(income_path)
print("sheets:", xls.sheet_names)

# まずは先頭シートを、生データとして覗く（ヘッダ未確定）
sheet = xls.sheet_names[0]
df_income_peek = pd.read_excel(income_path, sheet_name=sheet, header=None)

print("using sheet:", sheet)
display(df_income_peek.head(30))


sheets: ['FEH_00450091_251225093357']
using sheet: FEH_00450091_251225093357


,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,統計名：,賃金構造基本統計調査,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,表番号：,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,表題：,[令和２年以降] 一般_都道府県別_職種（特掲）DB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,実施年月：,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,市区町村時点（年月日）：,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,***,数字が得られないもの,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-,計数のない場合,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,X,統計精度上の理由により計数を表章することが不適当な場合,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# ==========================
# Step 15-2：必要行・必要列を固定
# ==========================

df_income = df_income_peek.copy()

# 列名を正式名称に置き換え（必要なものだけ）
df_income.columns = [
    "sex_code",
    "sex_sub",
    "sex",
    "job_code",
    "job_sub",
    "job_name",
    "time_code",
    "time_sub",
    "year",
    "region_code",
    *df_income.columns[10:]  # 後ろはそのまま
]

# 看護師・2023年・男女計だけに絞る
df_income = df_income[
    (df_income["job_name"] == "看護師") &
    (df_income["year"] == "2023年") &
    (df_income["sex"] == "男女計")
].copy()

print("rows:", len(df_income))
df_income[[
    "region_code",
    "job_name",
    "year"
]].head()


rows: 48


,region_code,job_name,year
13,0,看護師,2023年
14,1000,看護師,2023年
15,2000,看護師,2023年
16,3000,看護師,2023年
17,4000,看護師,2023年


In [56]:
df_income_peek.iloc[12]

0                性別_基本 コード
1              性別_基本 補助コード
2                    性別_基本
3       職種（小分類）（2020～） コード
4     職種（小分類）（2020～） 補助コード
5           職種（小分類）（2020～）
6       時間軸（2020～2023） コード
7     時間軸（2020～2023） 補助コード
8           時間軸（2020～2023）
9                   地域 コード
10                地域 補助コード
11                      地域
12                   /表章項目
13                   年齢【歳】
14                      注釈
15                 勤続年数【年】
16                      注釈
17           所定内実労働時間数【時間】
18                      注釈
19            超過実労働時間数【時間】
20                      注釈
21       きまって支給する現金給与額【千円】
22                      注釈
23              所定内給与額【千円】
24                      注釈
25        年間賞与その他特別給与額【千円】
26                      注釈
27                労働者数【十人】
28                      注釈
Name: 12, dtype: object

In [57]:
# ==========================
# Step 15-3：正式DataFrame作成（ヘッダ固定）
# ==========================

df_income = pd.read_excel(
    income_path,
    sheet_name=sheet,
    header=12   # ← ここが最重要
)

df_income.head()

,性別_基本 コード,性別_基本 補助コード,性別_基本,職種（小分類）（2020～） コード,職種（小分類）（2020～） 補助コード,職種（小分類）（2020～）,時間軸（2020～2023） コード,時間軸（2020～2023） 補助コード,時間軸（2020～2023）,地域 コード,...,超過実労働時間数【時間】,注釈.3,きまって支給する現金給与額【千円】,注釈.4,所定内給与額【千円】,注釈.5,年間賞与その他特別給与額【千円】,注釈.6,労働者数【十人】,注釈.7
0,1,NaN,男女計,1133,NaN,看護師,2023000000,NaN,2023年,0,...,6,NaN,352.1,NaN,319.3,NaN,856.5,NaN,83501,NaN
1,1,NaN,男女計,1133,NaN,看護師,2023000000,NaN,2023年,1000,...,5,NaN,334.0,NaN,311.4,NaN,781.0,NaN,4587,NaN
2,1,NaN,男女計,1133,NaN,看護師,2023000000,NaN,2023年,2000,...,5,NaN,301.6,NaN,274.3,NaN,733.0,NaN,821,NaN
3,1,NaN,男女計,1133,NaN,看護師,2023000000,NaN,2023年,3000,...,3,NaN,310.6,NaN,298.3,NaN,862.5,NaN,803,NaN
4,1,NaN,男女計,1133,NaN,看護師,2023000000,NaN,2023年,4000,...,6,NaN,365.8,NaN,320.0,NaN,968.4,NaN,1624,NaN


In [58]:
# ==========================
# Step 15-4：必要列抽出 → 47都道府県 → 年収算出（円）
# ==========================

import pandas as pd

# 必要列名（固定）
COL_REGION = "地域 コード"
COL_JOB = "職種（小分類）（2020～）"
COL_YEAR = "時間軸（2020～2023）"
COL_SEX = "性別_基本"

COL_AGE = "年齢【歳】"
COL_TENURE = "勤続年数【年】"
COL_OVERTIME = "超過実労働時間数【時間】"
COL_MONTHLY = "きまって支給する現金給与額【千円】"
COL_BONUS = "年間賞与その他特別給与額【千円】"

df_inc = df_income.copy()

# フィルタ（このシートは基本「看護師×2023×男女計」だが、念のため固定）
df_inc = df_inc[
    (df_inc[COL_JOB] == "看護師") &
    (df_inc[COL_YEAR] == "2023年") &
    (df_inc[COL_SEX] == "男女計")
].copy()

# 全国（地域コード=0）を除外
df_inc = df_inc[df_inc[COL_REGION] != 0].copy()

# 必要列だけ
df_inc = df_inc[[COL_REGION, COL_AGE, COL_TENURE, COL_OVERTIME, COL_MONTHLY, COL_BONUS]].copy()

# 数値化
for c in [COL_REGION, COL_AGE, COL_TENURE, COL_OVERTIME, COL_MONTHLY, COL_BONUS]:
    df_inc[c] = pd.to_numeric(df_inc[c], errors="coerce")

# 地域コード(1000,2000..) → 都道府県コード(1000=北海道 …) として扱う
# ここでは「code」を保持し、次ステップで prefecture 名に変換する
df_inc = df_inc.rename(columns={COL_REGION: "code"})

# 年収（円）: 月例給与×12 + 賞与（いずれも千円 → 円）
df_inc["nurse_income_annual_yen"] = (df_inc[COL_MONTHLY] * 12 + df_inc[COL_BONUS]) * 1000

# 列名を分析用に整える
df_income_out = df_inc.rename(columns={
    COL_AGE: "nurse_avg_age",
    COL_TENURE: "nurse_avg_tenure_years",
    COL_OVERTIME: "nurse_overtime_hours",
    COL_MONTHLY: "nurse_monthly_cash_k_yen",
    COL_BONUS: "nurse_bonus_k_yen",
}).copy()

# DQ
print("rows:", len(df_income_out))
print("dup code:", df_income_out["code"].duplicated().sum())
print("null annual:", df_income_out["nurse_income_annual_yen"].isna().sum())
print("null age:", df_income_out["nurse_avg_age"].isna().sum())
print("null tenure:", df_income_out["nurse_avg_tenure_years"].isna().sum())
print("null overtime:", df_income_out["nurse_overtime_hours"].isna().sum())

df_income_out.head()


rows: 47
dup code: 0
null annual: 0
null age: 0
null tenure: 0
null overtime: 0


,code,nurse_avg_age,nurse_avg_tenure_years,nurse_overtime_hours,nurse_monthly_cash_k_yen,nurse_bonus_k_yen,nurse_income_annual_yen
1,1000,42.0,8.0,5,334.0,781.0,4789000.0
2,2000,42.0,11.9,5,301.6,733.0,4352200.0
3,3000,46.0,11.8,3,310.6,862.5,4589700.0
4,4000,40.0,12.1,6,365.8,968.4,5358000.0
5,5000,44.9,14.0,4,345.2,1105.7,5248100.0


In [59]:
# ==========================
# Step 15-5：都道府県名（地域）を採用して整形（47行固定）
# ==========================

import re
import pandas as pd

# 「地域」列を使って prefecture を作る
df_income_out2 = df_inc.copy()  # Step15-4で作った df_inc（code列を持ってるやつ）を利用

# 念のため、地域列を追加で保持（df_inc作成時に落としていた場合に備える）
# df_income が元なので、同じフィルタをかけた行から「地域」を持ってくる
COL_REGION_NAME = "地域"
df_tmp = df_income[
    (df_income[COL_JOB] == "看護師") &
    (df_income[COL_YEAR] == "2023年") &
    (df_income[COL_SEX] == "男女計") &
    (df_income[COL_REGION] != 0)
].copy()

df_tmp = df_tmp[[COL_REGION, COL_REGION_NAME]].copy()
df_tmp[COL_REGION] = pd.to_numeric(df_tmp[COL_REGION], errors="coerce")
df_tmp = df_tmp.rename(columns={COL_REGION: "code", COL_REGION_NAME: "prefecture"})

# codeでマージしてprefecture付与
df_income_out2 = df_income_out.merge(df_tmp, on="code", how="left", validate="one_to_one")

# 都道府県名の正規化（余計な空白など除去）
df_income_out2["prefecture"] = (
    df_income_out2["prefecture"]
    .astype(str)
    .str.replace(r"\s+", "", regex=True)
)

# 47都道府県だけに限定（末尾が 都/道/府/県 のもの）
df_income_out2 = df_income_out2[
    df_income_out2["prefecture"].str.contains(r"(都|道|府|県)$", regex=True)
].copy()

# join用：prefectureキーで必要列だけ
df_income_out_final = df_income_out2[[
    "prefecture",
    "nurse_income_annual_yen",
    "nurse_overtime_hours",
    "nurse_avg_age",
    "nurse_avg_tenure_years",
    "nurse_monthly_cash_k_yen",
    "nurse_bonus_k_yen",
]].copy()

# DQ
print("rows:", len(df_income_out_final))
print("dup prefecture:", df_income_out_final["prefecture"].duplicated().sum())
print("null prefecture:", df_income_out_final["prefecture"].isna().sum())
print("null annual:", df_income_out_final["nurse_income_annual_yen"].isna().sum())

df_income_out_final.head()


rows: 47
dup prefecture: 0
null prefecture: 0
null annual: 0


/var/folders/y0/tqn1_kwn7wl40lxzswd0r1c40000gn/T/ipykernel_83233/2428510049.py:37: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_income_out2["prefecture"].str.contains(r"(都|道|府|県)$", regex=True)


,prefecture,nurse_income_annual_yen,nurse_overtime_hours,nurse_avg_age,nurse_avg_tenure_years,nurse_monthly_cash_k_yen,nurse_bonus_k_yen
0,北海道,4789000.0,5,42.0,8.0,334.0,781.0
1,青森県,4352200.0,5,42.0,11.9,301.6,733.0
2,岩手県,4589700.0,3,46.0,11.8,310.6,862.5
3,宮城県,5358000.0,6,40.0,12.1,365.8,968.4
4,秋田県,5248100.0,4,44.9,14.0,345.2,1105.7


In [60]:
# ==========================
# Step 15-6：df_masterへ年収関連を統合（冪等join）＋ checkpoint保存
# ==========================

from pathlib import Path

# 冪等：既存列があれば落としてから join
income_cols = [
    "nurse_income_annual_yen",
    "nurse_overtime_hours",
    "nurse_avg_age",
    "nurse_avg_tenure_years",
    "nurse_monthly_cash_k_yen",
    "nurse_bonus_k_yen",
]

drop_cols = [c for c in income_cols if c in df_master.columns]
if drop_cols:
    df_master = df_master.drop(columns=drop_cols)

before_rows = len(df_master)

df_master = df_master.merge(
    df_income_out_final,
    on="prefecture",
    how="left",
    validate="one_to_one"
)

after_rows = len(df_master)

print("before rows:", before_rows)
print("after rows:", after_rows)
for c in income_cols:
    print(f"null {c}:", df_master[c].isna().sum())

# checkpoint 保存
OUT_DIR = Path("../data/out")
out_path = OUT_DIR / "master_step8_medical_income.csv"
df_master.to_csv(out_path, index=False, encoding="utf-8-sig")

print("saved:", out_path)
df_master[["prefecture"] + income_cols].head()


before rows: 47
after rows: 47
null nurse_income_annual_yen: 0
null nurse_overtime_hours: 0
null nurse_avg_age: 0
null nurse_avg_tenure_years: 0
null nurse_monthly_cash_k_yen: 0
null nurse_bonus_k_yen: 0
saved: ../data/out/master_step8_medical_income.csv


,prefecture,nurse_income_annual_yen,nurse_overtime_hours,nurse_avg_age,nurse_avg_tenure_years,nurse_monthly_cash_k_yen,nurse_bonus_k_yen
0,北海道,4789000.0,5,42.0,8.0,334.0,781.0
1,青森県,4352200.0,5,42.0,11.9,301.6,733.0
2,岩手県,4589700.0,3,46.0,11.8,310.6,862.5
3,宮城県,5358000.0,6,40.0,12.1,365.8,968.4
4,秋田県,5248100.0,4,44.9,14.0,345.2,1105.7


In [61]:
# ==========================
# Step 16（REBUILD）：病院数（総数）＋人口10万対 を抽出
#  - メタ行が混ざる政府統計CSVに強い版
# ==========================

RAW_DIR = Path("../data/raw")
file_hospital = RAW_DIR / "厚労省_医療施設調査_病院数_2024.csv"

# --------------------------
# 0) まず先頭を覗いて「本当のヘッダ行」を探す
# --------------------------
df_peek = pd.read_csv(file_hospital, header=None, encoding="utf-8-sig", engine="python", nrows=80)

def find_header_row(df_peek: pd.DataFrame) -> int:
    """
    先頭n行から、都道府県列っぽい見出し & 年次列っぽい見出しが揃う行をヘッダとみなす。
    """
    for i in range(len(df_peek)):
        row = df_peek.iloc[i].astype(str).tolist()
        row_join = " ".join(row)

        # 「都道府県」っぽい列名と、「令和5」「人口10万」などが同じ行に出る行を探す
        cond_pref = ("都道府県" in row_join) or ("地域" in row_join)
        cond_year = ("令和5" in row_join) or ("2023" in row_join)
        cond_per = ("人口10万" in row_join) or ("10万" in row_join)

        if cond_pref and cond_year and cond_per:
            return i
    raise ValueError("❌ ヘッダ行を自動検出できませんでした。df_peek.head(30)を貼ってください。")

header_row = find_header_row(df_peek)
print("✅ detected header_row:", header_row)

# --------------------------
# 1) 本読み（検出したヘッダ行を使う）
# --------------------------
df_hospital_raw = pd.read_csv(
    file_hospital,
    header=header_row,
    encoding="utf-8-sig",
    engine="python"
)

print("▼ df_hospital_raw shape:", df_hospital_raw.shape)
print("▼ columns sample:", list(df_hospital_raw.columns)[:30])

# --------------------------
# 2) 必要列を「候補検索」で確実に拾う
# --------------------------
cols = [str(c) for c in df_hospital_raw.columns]

# 都道府県列（例：都道府県_005 / 都道府県 / 地域 など）
pref_candidates = [c for c in cols if ("都道府県" in c) or (c == "地域")]

# 「実数」列（令和5年(2023年) かつ 実数）
total_candidates = [c for c in cols if ("令和5" in c or "2023" in c) and ("実数" in c)]

# 「人口10万対」列
per100k_candidates = [c for c in cols if ("令和5" in c or "2023" in c) and ("人口10万" in c)]

print("pref_candidates:", pref_candidates)
print("total_candidates:", total_candidates)
print("per100k_candidates:", per100k_candidates)


✅ detected header_row: 14
▼ df_hospital_raw shape: (48, 20)
▼ columns sample: ['表章項目 コード', '表章項目 補助コード', '表章項目', '調査年10 コード', '調査年10 補助コード', '調査年10', '都道府県_005 コード', '都道府県_005 補助コード', '都道府県_005', '/年次_043(人口10万対）', '平成14年(2002年)', '平成17年(2005年)', '平成20年(2008年)', '平成23年(2011年)', '平成26年(2014年)', '平成29年(2017年)', '令和2年(2020年)', '令和4年(2022年)', '令和5年(2023年)（実数）', '令和5年(2023年)（人口10万対）']
pref_candidates: ['都道府県_005 コード', '都道府県_005 補助コード', '都道府県_005']
total_candidates: ['令和5年(2023年)（実数）']
per100k_candidates: ['令和5年(2023年)（人口10万対）']


In [62]:
# セル1：前提チェック
assert "df_hospital_raw" in globals(), "df_hospital_raw が未定義です（ヘッダ検出セルを先に実行してください）"

print("shape:", df_hospital_raw.shape)
print("columns:", list(df_hospital_raw.columns))


shape: (48, 20)
columns: ['表章項目 コード', '表章項目 補助コード', '表章項目', '調査年10 コード', '調査年10 補助コード', '調査年10', '都道府県_005 コード', '都道府県_005 補助コード', '都道府県_005', '/年次_043(人口10万対）', '平成14年(2002年)', '平成17年(2005年)', '平成20年(2008年)', '平成23年(2011年)', '平成26年(2014年)', '平成29年(2017年)', '令和2年(2020年)', '令和4年(2022年)', '令和5年(2023年)（実数）', '令和5年(2023年)（人口10万対）']


In [63]:
# セル2：列名確定（ログで確定していたやつ）
pref_col = "都道府県_005"
total_col = "令和5年(2023年)（実数）"
per100k_col = "令和5年(2023年)（人口10万対）"

use_cols = [pref_col, total_col, per100k_col]
missing = [c for c in use_cols if c not in df_hospital_raw.columns]
assert not missing, f"必要列がありません: {missing}"

df_hospital = df_hospital_raw[use_cols].copy()
df_hospital.head()


,都道府県_005,令和5年(2023年)（実数）,令和5年(2023年)（人口10万対）
0,全 国,"8,122",6.5
1,北海道,534,10.5
2,青 森,89,7.5
3,岩 手,91,7.8
4,宮 城,135,6.0


In [64]:
# セル3：都道府県整形＋全国除外
df_hospital[pref_col] = (
    df_hospital[pref_col]
    .astype(str)
    .str.replace("　", "", regex=False)
    .str.strip()
)

df_hospital = df_hospital[df_hospital[pref_col] != "全国"].copy()

print("rows:", len(df_hospital), "unique prefectures:", df_hospital[pref_col].nunique())
df_hospital.head()


rows: 47 unique prefectures: 47


,都道府県_005,令和5年(2023年)（実数）,令和5年(2023年)（人口10万対）
1,北海道,534,10.5
2,青森,89,7.5
3,岩手,91,7.8
4,宮城,135,6.0
5,秋田,64,7.0


In [65]:
# セル4：数値化
MISSING_MARKS = {"-": None, "...": None, "…": None, "***": None, "": None}

for c in [total_col, per100k_col]:
    df_hospital[c] = (
        df_hospital[c]
        .astype(str)
        .str.strip()
        .replace(MISSING_MARKS)
        .str.replace(",", "", regex=False)
    )
    df_hospital[c] = pd.to_numeric(df_hospital[c], errors="coerce")

df_hospital.isna().sum()


都道府県_005               0
令和5年(2023年)（実数）        0
令和5年(2023年)（人口10万対）    0
dtype: int64

In [66]:
# セル5：列名固定＋DQ
df_hospital = df_hospital.rename(columns={
    pref_col: "prefecture",
    total_col: "hospital_count_total",
    per100k_col: "hospital_count_per_100k",
})

# DQ
assert len(df_hospital) == 47, f"rows != 47: {len(df_hospital)}"
assert df_hospital["prefecture"].duplicated().sum() == 0, "prefecture が重複しています"
assert df_hospital["hospital_count_total"].isna().sum() == 0, "hospital_count_total に欠損があります"
assert df_hospital["hospital_count_per_100k"].isna().sum() == 0, "hospital_count_per_100k に欠損があります"

print("✅ Step16 PASSED")
df_hospital.head()


✅ Step16 PASSED


,prefecture,hospital_count_total,hospital_count_per_100k
1,北海道,534,10.5
2,青森,89,7.5
3,岩手,91,7.8
4,宮城,135,6.0
5,秋田,64,7.0


In [67]:
# セル6：保存（必要なら）
OUT_DIR = Path("../data/out")
OUT_DIR.mkdir(parents=True, exist_ok=True)

out_path = OUT_DIR / "master_step16_hospital_count.csv"
df_hospital.to_csv(out_path, index=False, encoding="utf-8-sig")

print("✅ saved:", out_path)


✅ saved: ../data/out/master_step16_hospital_count.csv


In [68]:
# セル6：保存（必要なら）
OUT_DIR = Path("../data/out")
OUT_DIR.mkdir(parents=True, exist_ok=True)

out_path = OUT_DIR / "master_step16_hospital_count.csv"
df_hospital.to_csv(out_path, index=False, encoding="utf-8-sig")

print("✅ saved:", out_path)


✅ saved: ../data/out/master_step16_hospital_count.csv


In [69]:
# ==========================
# Step 18-1（FIX4）：rawテキストで delimiter & header 行を特定
# ==========================

from pathlib import Path

RAW_DIR = Path("../data/raw")
file_t8 = RAW_DIR / "厚労省_医療施設調査_病床規模_2024.csv"

# 1) 先頭40行を「そのまま」表示（区切り文字を見る）
lines = []
with open(file_t8, "r", encoding="utf-8-sig", errors="replace") as f:
    for i in range(40):
        lines.append(f.readline())

print("▼ first 40 lines (raw)")
for i, line in enumerate(lines):
    print(f"{i:02d}: {line.rstrip()!r}")

# 2) 区切り文字の当たりをつける（カンマ/タブ/セミコロン）
def count_delims(s: str):
    return {
        "comma": s.count(","),
        "tab": s.count("\t"),
        "semi": s.count(";"),
    }

print("\n▼ delimiter counts (first 40 lines)")
for i, line in enumerate(lines):
    c = count_delims(line)
    # 目立つ行だけ表示（どれかが2以上）
    if max(c.values()) >= 2:
        print(i, c, line.rstrip()[:80])

# 3) 「表章項目」が含まれる行番号を探す（ヘッダ候補


▼ first 40 lines (raw)
00: '"統計名：","医療施設調査 令和５年医療施設（静態・動態）調査 都道府県編"'
01: '"表番号：","T8"'
02: '"表題：","第８表\u3000病院数，病院－病床の種類・病床の規模・都道府県－指定都市・特別区・中核市（再掲）別"'
03: '"実施年月：","2023年","-"'
04: ''
05: '"***","数字が得られないもの"'
06: '"-","計数のない場合"'
07: '"...","..."'
08: '"…","…"'
09: '"\u3000","E0020のCSV内の空欄"'
10: '"・","・"'
11: ''
12: '"","","","","","","","","","","","","/病床の種類_H25 コード","1","2","3","4","5","6","7","8","9","10","11","19","12","13","14","15","16","17","18"'
13: '"","","","","","","","","","","","","/病床の種類_H25 補助コード","-","-","-","-","-","-","-","-","-","-","-","-","-","-","-","-","-","-","-"'
14: '"表章項目 コード","表章項目 補助コード","表章項目","調査年10 コード","調査年10 補助コード","調査年10","都道府県－指定都市・特別区・中核市（再掲） コード","都道府県－指定都市・特別区・中核市（再掲） 補助コード","都道府県－指定都市・特別区・中核市（再掲）","病床の規模_001 コード","病床の規模_001 補助コード","病床の規模_001","/病床の種類_H25","総数","精神科病院","一般病院（総数）","一般病院（療養病床及び一般病床のみの病院）","一般病院（その他の一般病院（総数））","一般病院（その他の一般病院（精神病床））","一般病院（その他の一般病院（感染症病床））","一般病院（その他の一般病院（結核病床））","一般病院（その他の一般病院（療養病床））","一般病院（その他の一般病院（一般病床））","地域医療支援病

In [70]:
# ==========================
# Step 18-1（FIX 最終）：T8 正式ロード（skiprowsでメタ行除外）
# ==========================

from pathlib import Path
import pandas as pd

RAW_DIR = Path("../data/raw")
file_t8 = RAW_DIR / "厚労省_医療施設調査_病床規模_2024.csv"

df_t8_raw = pd.read_csv(
    file_t8,
    skiprows=14,          # ←ヘッダ行の直前まで捨てる（0-13を捨てる）
    header=0,             # ←行14がヘッダとして効く
    encoding="utf-8-sig",
    thousands=",",        # ← "8,122" を 8122 にできる
    na_values=["-", "***", "...", "…", ""]  # ←欠損の統一
)

print("▼ df_t8_raw loaded (fixed)")
print("rows:", len(df_t8_raw), "cols:", len(df_t8_raw.columns))
print("columns sample:", list(df_t8_raw.columns)[:8])

# 必須列の存在チェック
required_cols = [
    "表章項目",
    "都道府県－指定都市・特別区・中核市（再掲）",
    "病床の規模_001",
    "総数",
    "精神科病院",
    "一般病院（総数）",
]
missing = [c for c in required_cols if c not in df_t8_raw.columns]
print("missing required cols:", missing)

display(df_t8_raw[required_cols].head(10))


▼ df_t8_raw loaded (fixed)
rows: 1965 cols: 32
columns sample: ['表章項目 コード', '表章項目 補助コード', '表章項目', '調査年10 コード', '調査年10 補助コード', '調査年10', '都道府県－指定都市・特別区・中核市（再掲） コード', '都道府県－指定都市・特別区・中核市（再掲） 補助コード']
missing required cols: []


,表章項目,都道府県－指定都市・特別区・中核市（再掲）,病床の規模_001,総数,精神科病院,一般病院（総数）
0,病院数,全国,総数,8122.0,1057.0,7065.0
1,病院数,全国,20～ 29床,136.0,2.0,134.0
2,病院数,全国,30～ 39床,304.0,NaN,304.0
3,病院数,全国,40～ 49床,479.0,2.0,477.0
4,病院数,全国,50～ 99床,1997.0,43.0,1954.0
5,病院数,全国,100～149床,1410.0,193.0,1217.0
6,病院数,全国,150～199床,1378.0,256.0,1122.0
7,病院数,全国,200～299床,1013.0,335.0,678.0
8,病院数,全国,300～399床,670.0,141.0,529.0
9,病院数,全国,400～499床,354.0,59.0,295.0


In [71]:
# ==========================
# Step 18-2（FINAL FIX）：都道府県（47）× 病床規模=総数 を抽出
# ==========================

area_code_col = "都道府県－指定都市・特別区・中核市（再掲） コード"
area_name_col = "都道府県－指定都市・特別区・中核市（再掲）"
scale_code_col = "病床の規模_001 コード"

# 1) 病院数に限定
df_t8_hosp = df_t8_raw[df_t8_raw["表章項目"] == "病院数"].copy()

# 2) コード整形
df_t8_hosp[area_code_col] = df_t8_hosp[area_code_col].astype(str).str.zfill(5)
df_t8_hosp[scale_code_col] = pd.to_numeric(df_t8_hosp[scale_code_col], errors="coerce")

# 3) 病床規模=総数（コード=1）
df_t8_total = df_t8_hosp[df_t8_hosp[scale_code_col] == 1].copy()

# 4) 都道府県コードだけ（00110〜00570）
df_t8_pref = df_t8_total[
    (df_t8_total[area_code_col] >= "00110") &
    (df_t8_total[area_code_col] <= "00570")
].copy()

print("rows:", len(df_t8_pref), "unique prefectures:", df_t8_pref[area_code_col].nunique())
display(df_t8_pref[[area_code_col, area_name_col]].head(10))

assert len(df_t8_pref) == 47
assert df_t8_pref[area_code_col].nunique() == 47

# 5) 必要列抽出
df_hospital_type = df_t8_pref[[
    area_name_col,
    "総数",
    "精神科病院",
    "一般病院（総数）",
]].rename(columns={
    area_name_col: "prefecture_raw",
    "総数": "hospital_count_total_t8",
    "精神科病院": "psychiatric_hospital_count",
    "一般病院（総数）": "general_hospital_count",
})

# 6) 都道府県名を df_master 形式へ正規化（県/府/都/道）
def normalize_prefecture_to_jp_suffix(name: str) -> str:
    name = str(name).strip().replace("　", "")
    if name == "北海道":
        return "北海道"
    if name == "東京":
        return "東京都"
    if name == "大阪":
        return "大阪府"
    if name == "京都":
        return "京都府"
    if name.endswith(("都","道","府","県")):
        return name
    return name + "県"

df_hospital_type["prefecture"] = df_hospital_type["prefecture_raw"].map(normalize_prefecture_to_jp_suffix)

# 7) 数値化
for c in ["hospital_count_total_t8","psychiatric_hospital_count","general_hospital_count"]:
    df_hospital_type[c] = pd.to_numeric(df_hospital_type[c], errors="coerce")

# DQ
assert len(df_hospital_type) == 47
assert df_hospital_type["prefecture"].duplicated().sum() == 0
assert df_hospital_type[["hospital_count_total_t8","psychiatric_hospital_count","general_hospital_count"]].isna().sum().sum() == 0

# 検算
df_hospital_type["check_diff"] = (
    df_hospital_type["hospital_count_total_t8"]
    - (df_hospital_type["psychiatric_hospital_count"] + df_hospital_type["general_hospital_count"])
)
display(df_hospital_type["check_diff"].value_counts().head(10))

print("✅ Step 18-2 PASSED")
display(df_hospital_type.head())


rows: 47 unique prefectures: 47


,都道府県－指定都市・特別区・中核市（再掲） コード,都道府県－指定都市・特別区・中核市（再掲）
15,00110,北海道
30,00120,青森
45,00130,岩手
60,00140,宮城
75,00150,秋田
90,00160,山形
105,00170,福島
120,00180,茨城
135,00190,栃木
150,00200,群馬


check_diff
0.0    47
Name: count, dtype: int64

✅ Step 18-2 PASSED


,prefecture_raw,hospital_count_total_t8,psychiatric_hospital_count,general_hospital_count,prefecture,check_diff
15,北海道,534.0,70.0,464.0,北海道,0.0
30,青森,89.0,17.0,72.0,青森県,0.0
45,岩手,91.0,15.0,76.0,岩手県,0.0
60,宮城,135.0,27.0,108.0,宮城県,0.0
75,秋田,64.0,16.0,48.0,秋田県,0.0


In [73]:
# ==========================
# Step 18-3：T8から「大規模病院（>=500/700/900床）」を作成 → df_masterへ統合
# ==========================

from pathlib import Path
import pandas as pd

p = Path("../data/out/master_step10_population.csv")
if "population_total" not in df_master.columns:
    if p.exists():
        df_master = pd.read_csv(p)
        print("ℹ️ df_master を Step10 checkpoint から再ロードしました")
    else:
        raise FileNotFoundError(
            f"❌ {p} が存在しません。\n"
            "population_total が df_master に無いので per_100k を計算できません。\n"
            "対処: (1) 人口joinまで実行して df_master に population_total を入れる\n"
            "  または (2) 人口入りのmasterを data/out に保存して、そのパスをここに指定する"
        )

    print("ℹ️ df_master を Step10 checkpoint から再ロードしました")

area_code_col = "都道府県－指定都市・特別区・中核市（再掲） コード"
area_name_col = "都道府県－指定都市・特別区・中核市（再掲）"
scale_code_col = "病床の規模_001 コード"

# ------------------------------------------------------------
# 0) 前提チェック：人口がすでに df_master にある想定（population_total）
#    ※まだ無ければ、あなたの Step 17-α の join を先にやってください
# ------------------------------------------------------------
assert "population_total" in df_master.columns, "❌ df_master に population_total がありません（先に人口をjoinしてください）"

# ------------------------------------------------------------
# 1) T8：病院数だけに限定 & コード整形（あなたの Step18-2 と同じ）
# ------------------------------------------------------------
df_t8_hosp = df_t8_raw[df_t8_raw["表章項目"] == "病院数"].copy()

df_t8_hosp[area_code_col] = df_t8_hosp[area_code_col].astype(str).str.zfill(5)
df_t8_hosp[scale_code_col] = pd.to_numeric(df_t8_hosp[scale_code_col], errors="coerce")

# 都道府県だけ（全国・再掲の指定都市などを除外）
df_t8_hosp = df_t8_hosp[
    (df_t8_hosp[area_code_col] >= "00110") &
    (df_t8_hosp[area_code_col] <= "00570")
].copy()

# ------------------------------------------------------------
# 2) 病床規模コードのうち「500床以上」に該当するものを抽出
#    T8の並び（画像と一致）：
#      11=500～599床, 12=600～699床, 13=700～799床, 14=800～899床, 15=900床以上
# ------------------------------------------------------------
df_t8_large = df_t8_hosp[df_t8_hosp[scale_code_col].isin([11, 12, 13, 14, 15])].copy()

# 数値列（総数）を数値化（"1,234" や "-" 対応）
def to_num(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s in ["-", "...", "…", ""]:
        return np.nan
    return pd.to_numeric(s.replace(",", ""), errors="coerce")

df_t8_large["総数_num"] = df_t8_large["総数"].map(to_num)

# DQ：大規模側に欠損があるか（普通は無いはず。あったら原因調査が必要）
na_cnt = df_t8_large["総数_num"].isna().sum()
if na_cnt > 0:
    print("⚠️ large bins に NaN が混じっています（要確認）:", na_cnt)
    display(df_t8_large[df_t8_large["総数_num"].isna()][[area_code_col, area_name_col, scale_code_col, "総数"]].head(20))

# ------------------------------------------------------------
# 3) 都道府県名を df_master 形式へ正規化（あなたの関数を再利用）
# ------------------------------------------------------------
def normalize_prefecture_to_jp_suffix(name: str) -> str:
    name = str(name).strip().replace("　", "")
    if name == "北海道":
        return "北海道"
    if name == "東京":
        return "東京都"
    if name == "大阪":
        return "大阪府"
    if name == "京都":
        return "京都府"
    if name.endswith(("都","道","府","県")):
        return name
    return name + "県"

df_t8_large["prefecture"] = df_t8_large[area_name_col].map(normalize_prefecture_to_jp_suffix)

# ------------------------------------------------------------
# 4) 集計：500/700/900床以上の病院数を作る
#    - >=500: 11〜15 の合計
#    - >=700: 13〜15 の合計
#    - >=900: 15 のみ
# ------------------------------------------------------------
g = df_t8_large.groupby("prefecture")

df_large_counts = pd.DataFrame({
    "prefecture": sorted(df_t8_large["prefecture"].unique())
})

# ピボット（prefecture × scale_code）で作ってから足し算するのが安全
pv = df_t8_large.pivot_table(
    index="prefecture",
    columns=scale_code_col,
    values="総数_num",
    aggfunc="sum"
).fillna(0)

# 欲しい列が無い場合にも落ちないように get
def col(c):
    return pv[c] if c in pv.columns else 0

df_large_counts = pv.reset_index()[["prefecture"]].copy()
df_large_counts["large_hospital_500p_count"] = (col(11) + col(12) + col(13) + col(14) + col(15)).values
df_large_counts["large_hospital_700p_count"] = (col(13) + col(14) + col(15)).values
df_large_counts["mega_hospital_900p_count"]  = (col(15)).values

# 参考：サイズ帯ごとも残したい場合（任意）
df_large_counts["hosp_500_599_count"] = col(11).values
df_large_counts["hosp_600_699_count"] = col(12).values
df_large_counts["hosp_700_799_count"] = col(13).values
df_large_counts["hosp_800_899_count"] = col(14).values
df_large_counts["hosp_900p_count"]    = col(15).values

# DQ：47都道府県そろってるか
print("rows:", len(df_large_counts), "unique:", df_large_counts["prefecture"].nunique())
assert len(df_large_counts) == 47
assert df_large_counts["prefecture"].duplicated().sum() == 0
assert (df_large_counts[["large_hospital_500p_count","large_hospital_700p_count","mega_hospital_900p_count"]] >= 0).all().all()

# ------------------------------------------------------------
# 5) df_master と結合して per_100k を計算
# ------------------------------------------------------------
# 冪等：既存列があれば落としてから join
new_cols = [
    "large_hospital_500p_count",
    "large_hospital_700p_count",
    "mega_hospital_900p_count",
    "large_hospital_500p_per_100k",
    "large_hospital_700p_per_100k",
    "mega_hospital_900p_per_100k",
    # 参考：サイズ帯（任意）
    "hosp_500_599_count","hosp_600_699_count","hosp_700_799_count","hosp_800_899_count","hosp_900p_count"
]
drop_cols = [c for c in new_cols if c in df_master.columns]
if drop_cols:
    df_master = df_master.drop(columns=drop_cols)

before_rows = len(df_master)

df_master = df_master.merge(
    df_large_counts,
    on="prefecture",
    how="left",
    validate="one_to_one"
)

after_rows = len(df_master)
print("before rows:", before_rows)
print("after rows :", after_rows)
assert before_rows == 47
assert after_rows == 47

# missingチェック
missing = df_master.loc[df_master["large_hospital_500p_count"].isna(), "prefecture"].tolist()
print("▼ missing_prefectures (large hospital join)")
print(missing)
assert len(missing) == 0, f"❌ missing prefectures: {missing}"

# per_100k（人口10万人あたり）
df_master["large_hospital_500p_per_100k"] = df_master["large_hospital_500p_count"] / df_master["population_total"] * 100000
df_master["large_hospital_700p_per_100k"] = df_master["large_hospital_700p_count"] / df_master["population_total"] * 100000
df_master["mega_hospital_900p_per_100k"]  = df_master["mega_hospital_900p_count"]  / df_master["population_total"] * 100000

# DQ：レンジざっくり（負はありえない）
for c in ["large_hospital_500p_per_100k","large_hospital_700p_per_100k","mega_hospital_900p_per_100k"]:
    assert (df_master[c] >= 0).all(), f"❌ {c} has negative values"

print("▼ per_100k range check")
print(df_master[["large_hospital_500p_per_100k","large_hospital_700p_per_100k","mega_hospital_900p_per_100k"]].describe())

# ------------------------------------------------------------
# 6) psychiatric_ratio もここで作る（任意）
#    ※ Step18-2 の df_hospital_type が存在する前提
# ------------------------------------------------------------
if "psychiatric_hospital_count" in df_master.columns and "hospital_count_total_t8" in df_master.columns:
    df_master["psychiatric_ratio"] = df_master["psychiatric_hospital_count"] / df_master["hospital_count_total_t8"]
    # DQ
    assert df_master["psychiatric_ratio"].between(0, 1).all(), "❌ psychiatric_ratio out of [0,1]"
    print("✅ psychiatric_ratio added")
else:
    print("ℹ️ psychiatric_ratio はスキップ（df_masterに psychiatric_hospital_count / hospital_count_total_t8 が無い）")

# ------------------------------------------------------------
# 7) checkpoint保存
# ------------------------------------------------------------
OUT_DIR = Path("../data/out")
OUT_DIR.mkdir(parents=True, exist_ok=True)

out_path = OUT_DIR / "master_step11_large_hospitals.csv"
df_master.to_csv(out_path, index=False, encoding="utf-8-sig")
print("\n✅ saved:", out_path)

# 目視確認
display(df_master[[
    "prefecture",
    "large_hospital_500p_count", "large_hospital_700p_count", "mega_hospital_900p_count",
    "large_hospital_500p_per_100k", "large_hospital_700p_per_100k", "mega_hospital_900p_per_100k"
]].head(10))


FileNotFoundError: ❌ ../data/out/master_step10_population.csv が存在しません。
population_total が df_master に無いので per_100k を計算できません。
対処: (1) 人口joinまで実行して df_master に population_total を入れる
  または (2) 人口入りのmasterを data/out に保存して、そのパスをここに指定する